# Деревья решений и случайный лес

*1. Подключитесь к одному из наборов данных на Kaggle: Вариант 3: Bank marketing.*

*Разберитесь в том, как устроен ваш датасет и какова постановка задачи. Сделайте необходимый препроцессинг данных (удаление лишних признаков, кодирование категориальных признаков, логарифмирование целевой переменной, масштабирование).*

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import GridSearchCV

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Загрузка данных
df = pd.read_csv('/kaggle/input/bank-marketing-dataset/bank.csv')
df.head()

In [ ]:
df.keys()

In [ ]:
#df.info()

*Извлеките целевой признак (target). Какая из задач обучения с учителем рассматривается –– классификация или регрессия?*

Целевым признаком (target) является 'deposit', принимающий значения "yes" и "no", что означает положит клиент срочный депозит или нет. Рассматривается категория - обучение с учителем и тип задачи - классификация (предсказание категории объекта).


*Каково распределение значений target-переменной? Постройте подходящую визуализацию. Прокомментируйте результат.*

In [ ]:
import seaborn as sns
sns.catplot(x = "deposit", kind = "count", palette = "ch:.25", data = df)

In [ ]:
# Выведем процентное соотношение
df['deposit'].value_counts(normalize=True)

In [ ]:
# print(df['deposit'].shape) # Всего 11162 строк
df['deposit']

Имеем ~52,6% клиентов без депозитов и ~47,4% клиентов c депозитом.

Можно заметить, что классы 'no' и 'yes' почти сбалансированы.

*Проведите необходимую предобработку данных (preprocessing). Для построения моделей с помощью метрических методов все признаки должны быть закодированы числами. Полезными будут следующие методы библиотеки Pandas:*
1. map() – для перекодировки категориальной переменной числовыми метками; 
2. get_dummies() – для создания нескольких бинарных признаков на основе категориального. 

*Также может потребоваться масштабирование данных (scaling). Воспользуйтесь классом StandardScaler библиотеки Scikit-learn.*

In [ ]:
from sklearn. preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(df.default)
df['default_le'] = le.transform(df.default)

le_1 = LabelEncoder()
le_1.fit(df.housing)
df['housing_le'] = le_1.transform(df.housing)

le_2 = LabelEncoder()
le_2.fit(df.housing)
df['loan_le'] = le_2.transform(df.loan)

le_3 = LabelEncoder()
le_3.fit(df.housing)
df['deposit_le'] = le_3.transform(df.deposit)

dct_1 = {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6, 'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec':12}
df['month_le'] = df['month'].map(dct_1)

df.head().T

In [ ]:
df_1 = pd.read_csv('/kaggle/input/bank-marketing-dataset/bank.csv')
df_1['default'] = df['default_le']
df_1['housing'] = df['housing_le']
df_1['loan'] = df['loan_le']
df_1['deposit'] = df['deposit_le']
df_1['month'] = df['month_le']

# get_dummies
df_1 = pd.get_dummies(df_1, columns=['marital', 'education', 'contact', 'poutcome'])

df_1.head().T

In [ ]:
#df_1.info()

In [ ]:
#df_2 = df_1.drop(['job'], axis = 1)
df_2 = pd.get_dummies(df_1, columns=['job']) 

df_2.head().T

In [ ]:
#df_2.info()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Создание X - вся таблица без target (deposit), а y - target (deposit).

y = df_2['deposit']
df_3 = df_2.drop('deposit', axis = 1)

X = df_3

X_new = scaler.fit_transform(X)
X_new

*2. Разбейте набор данных на обучающую и валидационную (тестовую) выборки с помощью метода train_test_split.*

In [ ]:
# Разбиение
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, 
                                                      y, 
                                                      test_size=0.25, 
                                                      random_state=20) 

#random_state. Controls the shuffling applied to the data before applying the split. 

In [ ]:
print(X_train.shape, y_train.shape)
print( X_valid.shape, y_valid.shape)

## Обучение дерева решений и настройка гиперпараметров

*Обучите алгоритм <span style="color:green">классификации DecisionTrееClassifler</span> или регрессии DecisionTreeRegressor. Оцените качество каждой модели на отложенной выборке с помощью*
- <span style="color:green">*асcuracy_sсoге (либо другой подходящей метрики) для классификации*</span> 
- *mean_squared_error (либо другой подходящей метрики) для регрессии*

Обучим дерево решений с максимальной глубиной 3 и параметром random_state=2019.

In [ ]:
# Обучение дерева решений
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(max_depth = 3, random_state = 2019)
tree.fit(X_train, y_train)

Изобразим полученное дерево графически.

In [ ]:
# Визуализация
from sklearn.tree import export_graphviz

export_graphviz(tree, out_file='tree.dot', feature_names = X.columns)
print(open('tree.dot').read()) 

[Картинка на гугл диске](https://drive.google.com/file/d/1N-zGWeDLeP9ETHjbZorUZe8VZ2R_NZlP/view?usp=sharing)

[Ссылка на 
Graphviz Online](https://dreampuf.github.io/GraphvizOnline/#digraph%20Tree%20%7B%0D%0Anode%20%5Bshape%3Dbox%5D%20%3B%0D%0A0%20%5Blabel%3D%22duration%20%3C%3D%20362.5%5Cngini%20%3D%200.498%5Cnsamples%20%3D%208371%5Cnvalue%20%3D%20%5B4419%2C%203952%5D%22%5D%20%3B%0D%0A1%20%5Blabel%3D%22poutcome_success%20%3C%3D%200.5%5Cngini%20%3D%200.43%5Cnsamples%20%3D%205441%5Cnvalue%20%3D%20%5B3739%2C%201702%5D%22%5D%20%3B%0D%0A0%20-%3E%201%20%5Blabeldistance%3D2.5%2C%20labelangle%3D45%2C%20headlabel%3D%22True%22%5D%20%3B%0D%0A2%20%5Blabel%3D%22housing%20%3C%3D%200.5%5Cngini%20%3D%200.373%5Cnsamples%20%3D%204894%5Cnvalue%20%3D%20%5B3679%2C%201215%5D%22%5D%20%3B%0D%0A1%20-%3E%202%20%3B%0D%0A3%20%5Blabel%3D%22gini%20%3D%200.465%5Cnsamples%20%3D%202504%5Cnvalue%20%3D%20%5B1582%2C%20922%5D%22%5D%20%3B%0D%0A2%20-%3E%203%20%3B%0D%0A4%20%5Blabel%3D%22gini%20%3D%200.215%5Cnsamples%20%3D%202390%5Cnvalue%20%3D%20%5B2097%2C%20293%5D%22%5D%20%3B%0D%0A2%20-%3E%204%20%3B%0D%0A5%20%5Blabel%3D%22duration%20%3C%3D%2086.5%5Cngini%20%3D%200.195%5Cnsamples%20%3D%20547%5Cnvalue%20%3D%20%5B60%2C%20487%5D%22%5D%20%3B%0D%0A1%20-%3E%205%20%3B%0D%0A6%20%5Blabel%3D%22gini%20%3D%200.231%5Cnsamples%20%3D%2015%5Cnvalue%20%3D%20%5B13%2C%202%5D%22%5D%20%3B%0D%0A5%20-%3E%206%20%3B%0D%0A7%20%5Blabel%3D%22gini%20%3D%200.161%5Cnsamples%20%3D%20532%5Cnvalue%20%3D%20%5B47%2C%20485%5D%22%5D%20%3B%0D%0A5%20-%3E%207%20%3B%0D%0A8%20%5Blabel%3D%22duration%20%3C%3D%20613.5%5Cngini%20%3D%200.356%5Cnsamples%20%3D%202930%5Cnvalue%20%3D%20%5B680%2C%202250%5D%22%5D%20%3B%0D%0A0%20-%3E%208%20%5Blabeldistance%3D2.5%2C%20labelangle%3D-45%2C%20headlabel%3D%22False%22%5D%20%3B%0D%0A9%20%5Blabel%3D%22contact_unknown%20%3C%3D%200.5%5Cngini%20%3D%200.447%5Cnsamples%20%3D%201378%5Cnvalue%20%3D%20%5B464%2C%20914%5D%22%5D%20%3B%0D%0A8%20-%3E%209%20%3B%0D%0A10%20%5Blabel%3D%22gini%20%3D%200.396%5Cnsamples%20%3D%201136%5Cnvalue%20%3D%20%5B309%2C%20827%5D%22%5D%20%3B%0D%0A9%20-%3E%2010%20%3B%0D%0A11%20%5Blabel%3D%22gini%20%3D%200.461%5Cnsamples%20%3D%20242%5Cnvalue%20%3D%20%5B155%2C%2087%5D%22%5D%20%3B%0D%0A9%20-%3E%2011%20%3B%0D%0A12%20%5Blabel%3D%22duration%20%3C%3D%20794.5%5Cngini%20%3D%200.24%5Cnsamples%20%3D%201552%5Cnvalue%20%3D%20%5B216%2C%201336%5D%22%5D%20%3B%0D%0A8%20-%3E%2012%20%3B%0D%0A13%20%5Blabel%3D%22gini%20%3D%200.322%5Cnsamples%20%3D%20615%5Cnvalue%20%3D%20%5B124%2C%20491%5D%22%5D%20%3B%0D%0A12%20-%3E%2013%20%3B%0D%0A14%20%5Blabel%3D%22gini%20%3D%200.177%5Cnsamples%20%3D%20937%5Cnvalue%20%3D%20%5B92%2C%20845%5D%22%5D%20%3B%0D%0A12%20-%3E%2014%20%3B%0D%0A%7D%0D%0A)

Сделаем предсказание с помощью полученного дерева на валидационном множестве. Вычислим метрику accuracy на нём.

In [ ]:
# Предсказания для валидационного множества
from sklearn.metrics import accuracy_score

y_pred = tree.predict(X_valid)
accuracy_score(y_valid, y_pred)

*1. Создайте генератор разбиений, который перемешивает выборку перед созданием блоков (shuffle=True). Числаблоков n_spits paвно 5. Задайте таюже параметр гапdom_state для воспроизводимости результатов. Например: kf = KFold(n_splits=5, shuffle=True, random_state=42)*

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 5, shuffle = True, random_state = 42)

*2. Осуществите кроссвалидацию модели для подбора пиперпараметров:*
- *максимальная глубина дерева mах_depth* 
- *минимальное число объектов для разбиения во внутренней вершине min_samples_split*
- *минимальное число объектов в листе min_samples_leaf*
- *максимальное количество признаков, рассматриваемых при поиске лучшего разбиения max_features*

*Используйте GridsearchCV. Интервалы изменения гиперпараметров задайте самостоятельно.*

In [ ]:
# # Кросс-валидация и подбор гиперпараметров

# tree_params = {'max_depth': np.arange(2, 38),
#                 'min_samples_split': np.arange(2, 38), 
#                 'min_samples_leaf': np.arange(2, 38), 
#                 'max_features': np.arange(2, 38)
#               }

# tree_grid = GridSearchCV(tree, tree_params, cv=kf, scoring='accuracy')
# tree_grid.fit(X_train, y_train)

In [ ]:
# print("Estimator that was chosen by the search, i.e. estimator which gave highest score on the left out data:\n", 
#       tree_grid.best_estimator_)

In [ ]:
# print("Score of best_estimator on the left out data:", tree_grid.best_score_)

In [ ]:
# print("Parameter setting that gave the best results on the hold out data:\n", tree_grid.best_params_)

In [ ]:
tree_param_max_depth = {'max_depth': np.arange(2, 38)}

tree_grid_param_1 = GridSearchCV(tree, tree_param_max_depth, cv=kf, scoring='accuracy')
tree_grid_param_1.fit(X_train, y_train)

print(tree_grid_param_1.best_score_, "\n", tree_grid_param_1.best_params_)

In [ ]:
tree_param_min_samples_split = {'min_samples_split': np.arange(2, 38)}
tree_2 = DecisionTreeClassifier(max_depth = 15)
tree_grid_param_2 = GridSearchCV(tree_2, tree_param_min_samples_split, cv=kf, scoring='accuracy')
tree_grid_param_2.fit(X_train, y_train)

print(tree_grid_param_2.best_estimator_,"\n", tree_grid_param_2.best_score_, "\n", tree_grid_param_2.best_params_)

In [ ]:
tree_param_min_samples_leaf = {'min_samples_leaf': np.arange(2, 38)}
tree_3 = DecisionTreeClassifier(max_depth = 15, min_samples_split = 37)
tree_grid_param_3 = GridSearchCV(tree_3, tree_param_min_samples_leaf, cv=kf, scoring='accuracy')
tree_grid_param_3.fit(X_train, y_train)

print(tree_grid_param_3.best_estimator_,"\n", tree_grid_param_3.best_score_, "\n", tree_grid_param_3.best_params_)

In [ ]:
X_train.shape

In [ ]:
tree_param_max_features = {'max_features': np.arange(2, 38)}
tree_4 = DecisionTreeClassifier(max_depth = 15, min_samples_split = 37, min_samples_leaf = 13)
tree_grid_param_4 = GridSearchCV(tree_4, tree_param_max_features, cv=kf, scoring='accuracy')
tree_grid_param_4.fit(X_train, y_train)

print(tree_grid_param_4.best_estimator_,"\n", tree_grid_param_4.best_score_, "\n", tree_grid_param_4.best_params_)

*3. Постройте валидационные кривые (зависимость метрики от значений гиперпараметров). Сделайте выводы.*

In [ ]:
# Построим графики зависимости средней доли правильных ответов 
# (mean_test_score) от значений каждого из гиперпараметров.
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=2, sharey=True, figsize=(8,13))

ax[0, 0].plot(tree_param_max_depth['max_depth'], tree_grid_param_1.cv_results_['mean_test_score'])
ax[0, 0].set_xlabel('max_depth')
ax[0, 0].set_ylabel('Mean accuracy on test set')

ax[0, 1].plot(tree_param_min_samples_split['min_samples_split'], tree_grid_param_2.cv_results_['mean_test_score'])
ax[0, 1].set_xlabel('min_samples_split')
ax[0, 1].set_ylabel('Mean accuracy on test set')

ax[1, 0].plot(tree_param_min_samples_leaf['min_samples_leaf'], tree_grid_param_3.cv_results_['mean_test_score'])
ax[1, 0].set_xlabel('min_samples_leaf')
ax[1, 0].set_ylabel('Mean accuracy on test set')

ax[1, 1].plot(tree_param_max_features['max_features'], tree_grid_param_4.cv_results_['mean_test_score'])
ax[1, 1].set_xlabel('max_features')
ax[1, 1].set_ylabel('Mean accuracy on test set')

Результативный набор гиперпараметров:
max_depth=15, 
max_features=37, 
min_samples_leaf=13,
min_samples_split=37.

Набор хороший, но не лучший, так как не удалось одновременно подобрать все 4 гиперпараметра.

*4. Постройте графически полученное дерево.*

In [ ]:
best_tree = DecisionTreeClassifier(max_depth = 15, 
                                   max_features = 37, 
                                   min_samples_leaf = 13, 
                                   min_samples_split = 37)
best_tree.fit(X_train, y_train)

In [ ]:
export_graphviz(best_tree, out_file='best_tree.dot', feature_names=X.columns)
print(open('best_tree.dot').read()) 

[Картинка на гугл диске](https://drive.google.com/file/d/1sAsKxEz5tMXakRcfeAG9sOCN7Po2hQwM/view?usp=sharing)

[Ссылка на Graphviz Online](https://dreampuf.github.io/GraphvizOnline/#digraph%20Tree%20%7B%0D%0Anode%20%5Bshape%3Dbox%5D%20%3B%0D%0A0%20%5Blabel%3D%22duration%20%3C%3D%20362.5%5Cngini%20%3D%200.498%5Cnsamples%20%3D%208371%5Cnvalue%20%3D%20%5B4419%2C%203952%5D%22%5D%20%3B%0D%0A1%20%5Blabel%3D%22poutcome_success%20%3C%3D%200.5%5Cngini%20%3D%200.43%5Cnsamples%20%3D%205441%5Cnvalue%20%3D%20%5B3739%2C%201702%5D%22%5D%20%3B%0D%0A0%20-%3E%201%20%5Blabeldistance%3D2.5%2C%20labelangle%3D45%2C%20headlabel%3D%22True%22%5D%20%3B%0D%0A2%20%5Blabel%3D%22housing%20%3C%3D%200.5%5Cngini%20%3D%200.373%5Cnsamples%20%3D%204894%5Cnvalue%20%3D%20%5B3679%2C%201215%5D%22%5D%20%3B%0D%0A1%20-%3E%202%20%3B%0D%0A3%20%5Blabel%3D%22duration%20%3C%3D%20124.5%5Cngini%20%3D%200.465%5Cnsamples%20%3D%202504%5Cnvalue%20%3D%20%5B1582%2C%20922%5D%22%5D%20%3B%0D%0A2%20-%3E%203%20%3B%0D%0A4%20%5Blabel%3D%22month%20%3C%3D%204.5%5Cngini%20%3D%200.22%5Cnsamples%20%3D%20796%5Cnvalue%20%3D%20%5B696%2C%20100%5D%22%5D%20%3B%0D%0A3%20-%3E%204%20%3B%0D%0A5%20%5Blabel%3D%22month%20%3C%3D%202.5%5Cngini%20%3D%200.45%5Cnsamples%20%3D%20158%5Cnvalue%20%3D%20%5B104%2C%2054%5D%22%5D%20%3B%0D%0A4%20-%3E%205%20%3B%0D%0A6%20%5Blabel%3D%22duration%20%3C%3D%2076.5%5Cngini%20%3D%200.265%5Cnsamples%20%3D%20102%5Cnvalue%20%3D%20%5B86%2C%2016%5D%22%5D%20%3B%0D%0A5%20-%3E%206%20%3B%0D%0A7%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2048%5Cnvalue%20%3D%20%5B48%2C%200%5D%22%5D%20%3B%0D%0A6%20-%3E%207%20%3B%0D%0A8%20%5Blabel%3D%22day%20%3C%3D%2027.5%5Cngini%20%3D%200.417%5Cnsamples%20%3D%2054%5Cnvalue%20%3D%20%5B38%2C%2016%5D%22%5D%20%3B%0D%0A6%20-%3E%208%20%3B%0D%0A9%20%5Blabel%3D%22day%20%3C%3D%2010.0%5Cngini%20%3D%200.491%5Cnsamples%20%3D%2037%5Cnvalue%20%3D%20%5B21%2C%2016%5D%22%5D%20%3B%0D%0A8%20-%3E%209%20%3B%0D%0A10%20%5Blabel%3D%22gini%20%3D%200.236%5Cnsamples%20%3D%2022%5Cnvalue%20%3D%20%5B19%2C%203%5D%22%5D%20%3B%0D%0A9%20-%3E%2010%20%3B%0D%0A11%20%5Blabel%3D%22gini%20%3D%200.231%5Cnsamples%20%3D%2015%5Cnvalue%20%3D%20%5B2%2C%2013%5D%22%5D%20%3B%0D%0A9%20-%3E%2011%20%3B%0D%0A12%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2017%5Cnvalue%20%3D%20%5B17%2C%200%5D%22%5D%20%3B%0D%0A8%20-%3E%2012%20%3B%0D%0A13%20%5Blabel%3D%22campaign%20%3C%3D%202.5%5Cngini%20%3D%200.436%5Cnsamples%20%3D%2056%5Cnvalue%20%3D%20%5B18%2C%2038%5D%22%5D%20%3B%0D%0A5%20-%3E%2013%20%3B%0D%0A14%20%5Blabel%3D%22education_secondary%20%3C%3D%200.5%5Cngini%20%3D%200.387%5Cnsamples%20%3D%2042%5Cnvalue%20%3D%20%5B11%2C%2031%5D%22%5D%20%3B%0D%0A13%20-%3E%2014%20%3B%0D%0A15%20%5Blabel%3D%22gini%20%3D%200.461%5Cnsamples%20%3D%2025%5Cnvalue%20%3D%20%5B9%2C%2016%5D%22%5D%20%3B%0D%0A14%20-%3E%2015%20%3B%0D%0A16%20%5Blabel%3D%22gini%20%3D%200.208%5Cnsamples%20%3D%2017%5Cnvalue%20%3D%20%5B2%2C%2015%5D%22%5D%20%3B%0D%0A14%20-%3E%2016%20%3B%0D%0A17%20%5Blabel%3D%22gini%20%3D%200.5%5Cnsamples%20%3D%2014%5Cnvalue%20%3D%20%5B7%2C%207%5D%22%5D%20%3B%0D%0A13%20-%3E%2017%20%3B%0D%0A18%20%5Blabel%3D%22age%20%3C%3D%2029.5%5Cngini%20%3D%200.134%5Cnsamples%20%3D%20638%5Cnvalue%20%3D%20%5B592%2C%2046%5D%22%5D%20%3B%0D%0A4%20-%3E%2018%20%3B%0D%0A19%20%5Blabel%3D%22balance%20%3C%3D%2098.0%5Cngini%20%3D%200.436%5Cnsamples%20%3D%2053%5Cnvalue%20%3D%20%5B36%2C%2017%5D%22%5D%20%3B%0D%0A18%20-%3E%2019%20%3B%0D%0A20%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2017%5Cnvalue%20%3D%20%5B17%2C%200%5D%22%5D%20%3B%0D%0A19%20-%3E%2020%20%3B%0D%0A21%20%5Blabel%3D%22gini%20%3D%200.498%5Cnsamples%20%3D%2036%5Cnvalue%20%3D%20%5B19%2C%2017%5D%22%5D%20%3B%0D%0A19%20-%3E%2021%20%3B%0D%0A22%20%5Blabel%3D%22day%20%3C%3D%204.5%5Cngini%20%3D%200.094%5Cnsamples%20%3D%20585%5Cnvalue%20%3D%20%5B556%2C%2029%5D%22%5D%20%3B%0D%0A18%20-%3E%2022%20%3B%0D%0A23%20%5Blabel%3D%22gini%20%3D%200.48%5Cnsamples%20%3D%2025%5Cnvalue%20%3D%20%5B15%2C%2010%5D%22%5D%20%3B%0D%0A22%20-%3E%2023%20%3B%0D%0A24%20%5Blabel%3D%22month%20%3C%3D%208.5%5Cngini%20%3D%200.066%5Cnsamples%20%3D%20560%5Cnvalue%20%3D%20%5B541%2C%2019%5D%22%5D%20%3B%0D%0A22%20-%3E%2024%20%3B%0D%0A25%20%5Blabel%3D%22pdays%20%3C%3D%20102.0%5Cngini%20%3D%200.025%5Cnsamples%20%3D%20474%5Cnvalue%20%3D%20%5B468%2C%206%5D%22%5D%20%3B%0D%0A24%20-%3E%2025%20%3B%0D%0A26%20%5Blabel%3D%22education_tertiary%20%3C%3D%200.5%5Cngini%20%3D%200.017%5Cnsamples%20%3D%20461%5Cnvalue%20%3D%20%5B457%2C%204%5D%22%5D%20%3B%0D%0A25%20-%3E%2026%20%3B%0D%0A27%20%5Blabel%3D%22job_blue-collar%20%3C%3D%200.5%5Cngini%20%3D%200.006%5Cnsamples%20%3D%20308%5Cnvalue%20%3D%20%5B307%2C%201%5D%22%5D%20%3B%0D%0A26%20-%3E%2027%20%3B%0D%0A28%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%20242%5Cnvalue%20%3D%20%5B242%2C%200%5D%22%5D%20%3B%0D%0A27%20-%3E%2028%20%3B%0D%0A29%20%5Blabel%3D%22age%20%3C%3D%2052.5%5Cngini%20%3D%200.03%5Cnsamples%20%3D%2066%5Cnvalue%20%3D%20%5B65%2C%201%5D%22%5D%20%3B%0D%0A27%20-%3E%2029%20%3B%0D%0A30%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2051%5Cnvalue%20%3D%20%5B51%2C%200%5D%22%5D%20%3B%0D%0A29%20-%3E%2030%20%3B%0D%0A31%20%5Blabel%3D%22gini%20%3D%200.124%5Cnsamples%20%3D%2015%5Cnvalue%20%3D%20%5B14%2C%201%5D%22%5D%20%3B%0D%0A29%20-%3E%2031%20%3B%0D%0A32%20%5Blabel%3D%22balance%20%3C%3D%20237.5%5Cngini%20%3D%200.038%5Cnsamples%20%3D%20153%5Cnvalue%20%3D%20%5B150%2C%203%5D%22%5D%20%3B%0D%0A26%20-%3E%2032%20%3B%0D%0A33%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2067%5Cnvalue%20%3D%20%5B67%2C%200%5D%22%5D%20%3B%0D%0A32%20-%3E%2033%20%3B%0D%0A34%20%5Blabel%3D%22balance%20%3C%3D%20634.5%5Cngini%20%3D%200.067%5Cnsamples%20%3D%2086%5Cnvalue%20%3D%20%5B83%2C%203%5D%22%5D%20%3B%0D%0A32%20-%3E%2034%20%3B%0D%0A35%20%5Blabel%3D%22gini%20%3D%200.147%5Cnsamples%20%3D%2025%5Cnvalue%20%3D%20%5B23%2C%202%5D%22%5D%20%3B%0D%0A34%20-%3E%2035%20%3B%0D%0A36%20%5Blabel%3D%22day%20%3C%3D%2014.5%5Cngini%20%3D%200.032%5Cnsamples%20%3D%2061%5Cnvalue%20%3D%20%5B60%2C%201%5D%22%5D%20%3B%0D%0A34%20-%3E%2036%20%3B%0D%0A37%20%5Blabel%3D%22gini%20%3D%200.111%5Cnsamples%20%3D%2017%5Cnvalue%20%3D%20%5B16%2C%201%5D%22%5D%20%3B%0D%0A36%20-%3E%2037%20%3B%0D%0A38%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2044%5Cnvalue%20%3D%20%5B44%2C%200%5D%22%5D%20%3B%0D%0A36%20-%3E%2038%20%3B%0D%0A39%20%5Blabel%3D%22gini%20%3D%200.26%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B11%2C%202%5D%22%5D%20%3B%0D%0A25%20-%3E%2039%20%3B%0D%0A40%20%5Blabel%3D%22month%20%3C%3D%2010.5%5Cngini%20%3D%200.257%5Cnsamples%20%3D%2086%5Cnvalue%20%3D%20%5B73%2C%2013%5D%22%5D%20%3B%0D%0A24%20-%3E%2040%20%3B%0D%0A41%20%5Blabel%3D%22gini%20%3D%200.498%5Cnsamples%20%3D%2017%5Cnvalue%20%3D%20%5B8%2C%209%5D%22%5D%20%3B%0D%0A40%20-%3E%2041%20%3B%0D%0A42%20%5Blabel%3D%22day%20%3C%3D%2017.5%5Cngini%20%3D%200.109%5Cnsamples%20%3D%2069%5Cnvalue%20%3D%20%5B65%2C%204%5D%22%5D%20%3B%0D%0A40%20-%3E%2042%20%3B%0D%0A43%20%5Blabel%3D%22gini%20%3D%200.355%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B10%2C%203%5D%22%5D%20%3B%0D%0A42%20-%3E%2043%20%3B%0D%0A44%20%5Blabel%3D%22duration%20%3C%3D%20109.0%5Cngini%20%3D%200.035%5Cnsamples%20%3D%2056%5Cnvalue%20%3D%20%5B55%2C%201%5D%22%5D%20%3B%0D%0A42%20-%3E%2044%20%3B%0D%0A45%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2043%5Cnvalue%20%3D%20%5B43%2C%200%5D%22%5D%20%3B%0D%0A44%20-%3E%2045%20%3B%0D%0A46%20%5Blabel%3D%22gini%20%3D%200.142%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B12%2C%201%5D%22%5D%20%3B%0D%0A44%20-%3E%2046%20%3B%0D%0A47%20%5Blabel%3D%22contact_unknown%20%3C%3D%200.5%5Cngini%20%3D%200.499%5Cnsamples%20%3D%201708%5Cnvalue%20%3D%20%5B886%2C%20822%5D%22%5D%20%3B%0D%0A3%20-%3E%2047%20%3B%0D%0A48%20%5Blabel%3D%22balance%20%3C%3D%20223.5%5Cngini%20%3D%200.498%5Cnsamples%20%3D%201527%5Cnvalue%20%3D%20%5B718%2C%20809%5D%22%5D%20%3B%0D%0A47%20-%3E%2048%20%3B%0D%0A49%20%5Blabel%3D%22month%20%3C%3D%206.5%5Cngini%20%3D%200.436%5Cnsamples%20%3D%20433%5Cnvalue%20%3D%20%5B294%2C%20139%5D%22%5D%20%3B%0D%0A48%20-%3E%2049%20%3B%0D%0A50%20%5Blabel%3D%22month%20%3C%3D%202.5%5Cngini%20%3D%200.5%5Cnsamples%20%3D%20156%5Cnvalue%20%3D%20%5B76%2C%2080%5D%22%5D%20%3B%0D%0A49%20-%3E%2050%20%3B%0D%0A51%20%5Blabel%3D%22day%20%3C%3D%2023.5%5Cngini%20%3D%200.386%5Cnsamples%20%3D%2065%5Cnvalue%20%3D%20%5B48%2C%2017%5D%22%5D%20%3B%0D%0A50%20-%3E%2051%20%3B%0D%0A52%20%5Blabel%3D%22day%20%3C%3D%205.5%5Cngini%20%3D%200.472%5Cnsamples%20%3D%2042%5Cnvalue%20%3D%20%5B26%2C%2016%5D%22%5D%20%3B%0D%0A51%20-%3E%2052%20%3B%0D%0A53%20%5Blabel%3D%22gini%20%3D%200.26%5Cnsamples%20%3D%2026%5Cnvalue%20%3D%20%5B22%2C%204%5D%22%5D%20%3B%0D%0A52%20-%3E%2053%20%3B%0D%0A54%20%5Blabel%3D%22gini%20%3D%200.375%5Cnsamples%20%3D%2016%5Cnvalue%20%3D%20%5B4%2C%2012%5D%22%5D%20%3B%0D%0A52%20-%3E%2054%20%3B%0D%0A55%20%5Blabel%3D%22gini%20%3D%200.083%5Cnsamples%20%3D%2023%5Cnvalue%20%3D%20%5B22%2C%201%5D%22%5D%20%3B%0D%0A51%20-%3E%2055%20%3B%0D%0A56%20%5Blabel%3D%22month%20%3C%3D%204.5%5Cngini%20%3D%200.426%5Cnsamples%20%3D%2091%5Cnvalue%20%3D%20%5B28%2C%2063%5D%22%5D%20%3B%0D%0A50%20-%3E%2056%20%3B%0D%0A57%20%5Blabel%3D%22marital_married%20%3C%3D%200.5%5Cngini%20%3D%200.201%5Cnsamples%20%3D%2044%5Cnvalue%20%3D%20%5B5%2C%2039%5D%22%5D%20%3B%0D%0A56%20-%3E%2057%20%3B%0D%0A58%20%5Blabel%3D%22gini%20%3D%200.08%5Cnsamples%20%3D%2024%5Cnvalue%20%3D%20%5B1%2C%2023%5D%22%5D%20%3B%0D%0A57%20-%3E%2058%20%3B%0D%0A59%20%5Blabel%3D%22gini%20%3D%200.32%5Cnsamples%20%3D%2020%5Cnvalue%20%3D%20%5B4%2C%2016%5D%22%5D%20%3B%0D%0A57%20-%3E%2059%20%3B%0D%0A60%20%5Blabel%3D%22poutcome_failure%20%3C%3D%200.5%5Cngini%20%3D%200.5%5Cnsamples%20%3D%2047%5Cnvalue%20%3D%20%5B23%2C%2024%5D%22%5D%20%3B%0D%0A56%20-%3E%2060%20%3B%0D%0A61%20%5Blabel%3D%22gini%20%3D%200.457%5Cnsamples%20%3D%2034%5Cnvalue%20%3D%20%5B12%2C%2022%5D%22%5D%20%3B%0D%0A60%20-%3E%2061%20%3B%0D%0A62%20%5Blabel%3D%22gini%20%3D%200.26%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B11%2C%202%5D%22%5D%20%3B%0D%0A60%20-%3E%2062%20%3B%0D%0A63%20%5Blabel%3D%22month%20%3C%3D%208.5%5Cngini%20%3D%200.335%5Cnsamples%20%3D%20277%5Cnvalue%20%3D%20%5B218%2C%2059%5D%22%5D%20%3B%0D%0A49%20-%3E%2063%20%3B%0D%0A64%20%5Blabel%3D%22previous%20%3C%3D%200.5%5Cngini%20%3D%200.216%5Cnsamples%20%3D%20219%5Cnvalue%20%3D%20%5B192%2C%2027%5D%22%5D%20%3B%0D%0A63%20-%3E%2064%20%3B%0D%0A65%20%5Blabel%3D%22age%20%3C%3D%2056.5%5Cngini%20%3D%200.159%5Cnsamples%20%3D%20206%5Cnvalue%20%3D%20%5B188%2C%2018%5D%22%5D%20%3B%0D%0A64%20-%3E%2065%20%3B%0D%0A66%20%5Blabel%3D%22age%20%3C%3D%2027.5%5Cngini%20%3D%200.111%5Cnsamples%20%3D%20186%5Cnvalue%20%3D%20%5B175%2C%2011%5D%22%5D%20%3B%0D%0A65%20-%3E%2066%20%3B%0D%0A67%20%5Blabel%3D%22gini%20%3D%200.444%5Cnsamples%20%3D%2018%5Cnvalue%20%3D%20%5B12%2C%206%5D%22%5D%20%3B%0D%0A66%20-%3E%2067%20%3B%0D%0A68%20%5Blabel%3D%22day%20%3C%3D%207.5%5Cngini%20%3D%200.058%5Cnsamples%20%3D%20168%5Cnvalue%20%3D%20%5B163%2C%205%5D%22%5D%20%3B%0D%0A66%20-%3E%2068%20%3B%0D%0A69%20%5Blabel%3D%22gini%20%3D%200.278%5Cnsamples%20%3D%2024%5Cnvalue%20%3D%20%5B20%2C%204%5D%22%5D%20%3B%0D%0A68%20-%3E%2069%20%3B%0D%0A70%20%5Blabel%3D%22campaign%20%3C%3D%201.5%5Cngini%20%3D%200.014%5Cnsamples%20%3D%20144%5Cnvalue%20%3D%20%5B143%2C%201%5D%22%5D%20%3B%0D%0A68%20-%3E%2070%20%3B%0D%0A71%20%5Blabel%3D%22gini%20%3D%200.083%5Cnsamples%20%3D%2023%5Cnvalue%20%3D%20%5B22%2C%201%5D%22%5D%20%3B%0D%0A70%20-%3E%2071%20%3B%0D%0A72%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%20121%5Cnvalue%20%3D%20%5B121%2C%200%5D%22%5D%20%3B%0D%0A70%20-%3E%2072%20%3B%0D%0A73%20%5Blabel%3D%22gini%20%3D%200.455%5Cnsamples%20%3D%2020%5Cnvalue%20%3D%20%5B13%2C%207%5D%22%5D%20%3B%0D%0A65%20-%3E%2073%20%3B%0D%0A74%20%5Blabel%3D%22gini%20%3D%200.426%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B4%2C%209%5D%22%5D%20%3B%0D%0A64%20-%3E%2074%20%3B%0D%0A75%20%5Blabel%3D%22month%20%3C%3D%2010.5%5Cngini%20%3D%200.495%5Cnsamples%20%3D%2058%5Cnvalue%20%3D%20%5B26%2C%2032%5D%22%5D%20%3B%0D%0A63%20-%3E%2075%20%3B%0D%0A76%20%5Blabel%3D%22gini%20%3D%200.328%5Cnsamples%20%3D%2029%5Cnvalue%20%3D%20%5B6%2C%2023%5D%22%5D%20%3B%0D%0A75%20-%3E%2076%20%3B%0D%0A77%20%5Blabel%3D%22gini%20%3D%200.428%5Cnsamples%20%3D%2029%5Cnvalue%20%3D%20%5B20%2C%209%5D%22%5D%20%3B%0D%0A75%20-%3E%2077%20%3B%0D%0A78%20%5Blabel%3D%22pdays%20%3C%3D%2024.5%5Cngini%20%3D%200.475%5Cnsamples%20%3D%201094%5Cnvalue%20%3D%20%5B424%2C%20670%5D%22%5D%20%3B%0D%0A48%20-%3E%2078%20%3B%0D%0A79%20%5Blabel%3D%22month%20%3C%3D%206.5%5Cngini%20%3D%200.493%5Cnsamples%20%3D%20858%5Cnvalue%20%3D%20%5B377%2C%20481%5D%22%5D%20%3B%0D%0A78%20-%3E%2079%20%3B%0D%0A80%20%5Blabel%3D%22month%20%3C%3D%202.5%5Cngini%20%3D%200.413%5Cnsamples%20%3D%20408%5Cnvalue%20%3D%20%5B119%2C%20289%5D%22%5D%20%3B%0D%0A79%20-%3E%2080%20%3B%0D%0A81%20%5Blabel%3D%22day%20%3C%3D%2028.5%5Cngini%20%3D%200.5%5Cnsamples%20%3D%20145%5Cnvalue%20%3D%20%5B72%2C%2073%5D%22%5D%20%3B%0D%0A80%20-%3E%2081%20%3B%0D%0A82%20%5Blabel%3D%22day%20%3C%3D%209.5%5Cngini%20%3D%200.491%5Cnsamples%20%3D%20129%5Cnvalue%20%3D%20%5B56%2C%2073%5D%22%5D%20%3B%0D%0A81%20-%3E%2082%20%3B%0D%0A83%20%5Blabel%3D%22education_secondary%20%3C%3D%200.5%5Cngini%20%3D%200.47%5Cnsamples%20%3D%2074%5Cnvalue%20%3D%20%5B46%2C%2028%5D%22%5D%20%3B%0D%0A82%20-%3E%2083%20%3B%0D%0A84%20%5Blabel%3D%22age%20%3C%3D%2048.0%5Cngini%20%3D%200.499%5Cnsamples%20%3D%2042%5Cnvalue%20%3D%20%5B20%2C%2022%5D%22%5D%20%3B%0D%0A83%20-%3E%2084%20%3B%0D%0A85%20%5Blabel%3D%22gini%20%3D%200.48%5Cnsamples%20%3D%2025%5Cnvalue%20%3D%20%5B15%2C%2010%5D%22%5D%20%3B%0D%0A84%20-%3E%2085%20%3B%0D%0A86%20%5Blabel%3D%22gini%20%3D%200.415%5Cnsamples%20%3D%2017%5Cnvalue%20%3D%20%5B5%2C%2012%5D%22%5D%20%3B%0D%0A84%20-%3E%2086%20%3B%0D%0A87%20%5Blabel%3D%22gini%20%3D%200.305%5Cnsamples%20%3D%2032%5Cnvalue%20%3D%20%5B26%2C%206%5D%22%5D%20%3B%0D%0A83%20-%3E%2087%20%3B%0D%0A88%20%5Blabel%3D%22month%20%3C%3D%201.5%5Cngini%20%3D%200.298%5Cnsamples%20%3D%2055%5Cnvalue%20%3D%20%5B10%2C%2045%5D%22%5D%20%3B%0D%0A82%20-%3E%2088%20%3B%0D%0A89%20%5Blabel%3D%22gini%20%3D%200.496%5Cnsamples%20%3D%2022%5Cnvalue%20%3D%20%5B10%2C%2012%5D%22%5D%20%3B%0D%0A88%20-%3E%2089%20%3B%0D%0A90%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2033%5Cnvalue%20%3D%20%5B0%2C%2033%5D%22%5D%20%3B%0D%0A88%20-%3E%2090%20%3B%0D%0A91%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2016%5Cnvalue%20%3D%20%5B16%2C%200%5D%22%5D%20%3B%0D%0A81%20-%3E%2091%20%3B%0D%0A92%20%5Blabel%3D%22day%20%3C%3D%2025.5%5Cngini%20%3D%200.294%5Cnsamples%20%3D%20263%5Cnvalue%20%3D%20%5B47%2C%20216%5D%22%5D%20%3B%0D%0A80%20-%3E%2092%20%3B%0D%0A93%20%5Blabel%3D%22duration%20%3C%3D%20326.0%5Cngini%20%3D%200.248%5Cnsamples%20%3D%20214%5Cnvalue%20%3D%20%5B31%2C%20183%5D%22%5D%20%3B%0D%0A92%20-%3E%2093%20%3B%0D%0A94%20%5Blabel%3D%22day%20%3C%3D%2018.5%5Cngini%20%3D%200.27%5Cnsamples%20%3D%20193%5Cnvalue%20%3D%20%5B31%2C%20162%5D%22%5D%20%3B%0D%0A93%20-%3E%2094%20%3B%0D%0A95%20%5Blabel%3D%22day%20%3C%3D%2015.5%5Cngini%20%3D%200.311%5Cnsamples%20%3D%20140%5Cnvalue%20%3D%20%5B27%2C%20113%5D%22%5D%20%3B%0D%0A94%20-%3E%2095%20%3B%0D%0A96%20%5Blabel%3D%22day%20%3C%3D%204.5%5Cngini%20%3D%200.267%5Cnsamples%20%3D%20120%5Cnvalue%20%3D%20%5B19%2C%20101%5D%22%5D%20%3B%0D%0A95%20-%3E%2096%20%3B%0D%0A97%20%5Blabel%3D%22month%20%3C%3D%205.5%5Cngini%20%3D%200.381%5Cnsamples%20%3D%2043%5Cnvalue%20%3D%20%5B11%2C%2032%5D%22%5D%20%3B%0D%0A96%20-%3E%2097%20%3B%0D%0A98%20%5Blabel%3D%22gini%20%3D%200.49%5Cnsamples%20%3D%2014%5Cnvalue%20%3D%20%5B6%2C%208%5D%22%5D%20%3B%0D%0A97%20-%3E%2098%20%3B%0D%0A99%20%5Blabel%3D%22gini%20%3D%200.285%5Cnsamples%20%3D%2029%5Cnvalue%20%3D%20%5B5%2C%2024%5D%22%5D%20%3B%0D%0A97%20-%3E%2099%20%3B%0D%0A100%20%5Blabel%3D%22duration%20%3C%3D%20266.5%5Cngini%20%3D%200.186%5Cnsamples%20%3D%2077%5Cnvalue%20%3D%20%5B8%2C%2069%5D%22%5D%20%3B%0D%0A96%20-%3E%20100%20%3B%0D%0A101%20%5Blabel%3D%22gini%20%3D%200.098%5Cnsamples%20%3D%2058%5Cnvalue%20%3D%20%5B3%2C%2055%5D%22%5D%20%3B%0D%0A100%20-%3E%20101%20%3B%0D%0A102%20%5Blabel%3D%22gini%20%3D%200.388%5Cnsamples%20%3D%2019%5Cnvalue%20%3D%20%5B5%2C%2014%5D%22%5D%20%3B%0D%0A100%20-%3E%20102%20%3B%0D%0A103%20%5Blabel%3D%22gini%20%3D%200.48%5Cnsamples%20%3D%2020%5Cnvalue%20%3D%20%5B8%2C%2012%5D%22%5D%20%3B%0D%0A95%20-%3E%20103%20%3B%0D%0A104%20%5Blabel%3D%22age%20%3C%3D%2047.0%5Cngini%20%3D%200.14%5Cnsamples%20%3D%2053%5Cnvalue%20%3D%20%5B4%2C%2049%5D%22%5D%20%3B%0D%0A94%20-%3E%20104%20%3B%0D%0A105%20%5Blabel%3D%22gini%20%3D%200.219%5Cnsamples%20%3D%2032%5Cnvalue%20%3D%20%5B4%2C%2028%5D%22%5D%20%3B%0D%0A104%20-%3E%20105%20%3B%0D%0A106%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2021%5Cnvalue%20%3D%20%5B0%2C%2021%5D%22%5D%20%3B%0D%0A104%20-%3E%20106%20%3B%0D%0A107%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2021%5Cnvalue%20%3D%20%5B0%2C%2021%5D%22%5D%20%3B%0D%0A93%20-%3E%20107%20%3B%0D%0A108%20%5Blabel%3D%22duration%20%3C%3D%20196.5%5Cngini%20%3D%200.44%5Cnsamples%20%3D%2049%5Cnvalue%20%3D%20%5B16%2C%2033%5D%22%5D%20%3B%0D%0A92%20-%3E%20108%20%3B%0D%0A109%20%5Blabel%3D%22gini%20%3D%200.49%5Cnsamples%20%3D%2021%5Cnvalue%20%3D%20%5B12%2C%209%5D%22%5D%20%3B%0D%0A108%20-%3E%20109%20%3B%0D%0A110%20%5Blabel%3D%22gini%20%3D%200.245%5Cnsamples%20%3D%2028%5Cnvalue%20%3D%20%5B4%2C%2024%5D%22%5D%20%3B%0D%0A108%20-%3E%20110%20%3B%0D%0A111%20%5Blabel%3D%22campaign%20%3C%3D%201.5%5Cngini%20%3D%200.489%5Cnsamples%20%3D%20450%5Cnvalue%20%3D%20%5B258%2C%20192%5D%22%5D%20%3B%0D%0A79%20-%3E%20111%20%3B%0D%0A112%20%5Blabel%3D%22loan%20%3C%3D%200.5%5Cngini%20%3D%200.474%5Cnsamples%20%3D%20189%5Cnvalue%20%3D%20%5B73%2C%20116%5D%22%5D%20%3B%0D%0A111%20-%3E%20112%20%3B%0D%0A113%20%5Blabel%3D%22age%20%3C%3D%2029.5%5Cngini%20%3D%200.444%5Cnsamples%20%3D%20168%5Cnvalue%20%3D%20%5B56%2C%20112%5D%22%5D%20%3B%0D%0A112%20-%3E%20113%20%3B%0D%0A114%20%5Blabel%3D%22gini%20%3D%200.064%5Cnsamples%20%3D%2030%5Cnvalue%20%3D%20%5B1%2C%2029%5D%22%5D%20%3B%0D%0A113%20-%3E%20114%20%3B%0D%0A115%20%5Blabel%3D%22duration%20%3C%3D%20183.0%5Cngini%20%3D%200.479%5Cnsamples%20%3D%20138%5Cnvalue%20%3D%20%5B55%2C%2083%5D%22%5D%20%3B%0D%0A113%20-%3E%20115%20%3B%0D%0A116%20%5Blabel%3D%22month%20%3C%3D%208.5%5Cngini%20%3D%200.484%5Cnsamples%20%3D%2039%5Cnvalue%20%3D%20%5B23%2C%2016%5D%22%5D%20%3B%0D%0A115%20-%3E%20116%20%3B%0D%0A117%20%5Blabel%3D%22gini%20%3D%200.291%5Cnsamples%20%3D%2017%5Cnvalue%20%3D%20%5B14%2C%203%5D%22%5D%20%3B%0D%0A116%20-%3E%20117%20%3B%0D%0A118%20%5Blabel%3D%22gini%20%3D%200.483%5Cnsamples%20%3D%2022%5Cnvalue%20%3D%20%5B9%2C%2013%5D%22%5D%20%3B%0D%0A116%20-%3E%20118%20%3B%0D%0A119%20%5Blabel%3D%22day%20%3C%3D%2018.5%5Cngini%20%3D%200.438%5Cnsamples%20%3D%2099%5Cnvalue%20%3D%20%5B32%2C%2067%5D%22%5D%20%3B%0D%0A115%20-%3E%20119%20%3B%0D%0A120%20%5Blabel%3D%22duration%20%3C%3D%20260.0%5Cngini%20%3D%200.354%5Cnsamples%20%3D%2061%5Cnvalue%20%3D%20%5B14%2C%2047%5D%22%5D%20%3B%0D%0A119%20-%3E%20120%20%3B%0D%0A121%20%5Blabel%3D%22gini%20%3D%200.191%5Cnsamples%20%3D%2028%5Cnvalue%20%3D%20%5B3%2C%2025%5D%22%5D%20%3B%0D%0A120%20-%3E%20121%20%3B%0D%0A122%20%5Blabel%3D%22gini%20%3D%200.444%5Cnsamples%20%3D%2033%5Cnvalue%20%3D%20%5B11%2C%2022%5D%22%5D%20%3B%0D%0A120%20-%3E%20122%20%3B%0D%0A123%20%5Blabel%3D%22month%20%3C%3D%2010.5%5Cngini%20%3D%200.499%5Cnsamples%20%3D%2038%5Cnvalue%20%3D%20%5B18%2C%2020%5D%22%5D%20%3B%0D%0A119%20-%3E%20123%20%3B%0D%0A124%20%5Blabel%3D%22gini%20%3D%200.444%5Cnsamples%20%3D%2024%5Cnvalue%20%3D%20%5B8%2C%2016%5D%22%5D%20%3B%0D%0A123%20-%3E%20124%20%3B%0D%0A125%20%5Blabel%3D%22gini%20%3D%200.408%5Cnsamples%20%3D%2014%5Cnvalue%20%3D%20%5B10%2C%204%5D%22%5D%20%3B%0D%0A123%20-%3E%20125%20%3B%0D%0A126%20%5Blabel%3D%22gini%20%3D%200.308%5Cnsamples%20%3D%2021%5Cnvalue%20%3D%20%5B17%2C%204%5D%22%5D%20%3B%0D%0A112%20-%3E%20126%20%3B%0D%0A127%20%5Blabel%3D%22age%20%3C%3D%2059.5%5Cngini%20%3D%200.413%5Cnsamples%20%3D%20261%5Cnvalue%20%3D%20%5B185%2C%2076%5D%22%5D%20%3B%0D%0A111%20-%3E%20127%20%3B%0D%0A128%20%5Blabel%3D%22age%20%3C%3D%2029.5%5Cngini%20%3D%200.372%5Cnsamples%20%3D%20239%5Cnvalue%20%3D%20%5B180%2C%2059%5D%22%5D%20%3B%0D%0A127%20-%3E%20128%20%3B%0D%0A129%20%5Blabel%3D%22gini%20%3D%200.363%5Cnsamples%20%3D%2021%5Cnvalue%20%3D%20%5B5%2C%2016%5D%22%5D%20%3B%0D%0A128%20-%3E%20129%20%3B%0D%0A130%20%5Blabel%3D%22month%20%3C%3D%208.5%5Cngini%20%3D%200.317%5Cnsamples%20%3D%20218%5Cnvalue%20%3D%20%5B175%2C%2043%5D%22%5D%20%3B%0D%0A128%20-%3E%20130%20%3B%0D%0A131%20%5Blabel%3D%22duration%20%3C%3D%20319.5%5Cngini%20%3D%200.249%5Cnsamples%20%3D%20178%5Cnvalue%20%3D%20%5B152%2C%2026%5D%22%5D%20%3B%0D%0A130%20-%3E%20131%20%3B%0D%0A132%20%5Blabel%3D%22day%20%3C%3D%207.5%5Cngini%20%3D%200.208%5Cnsamples%20%3D%20161%5Cnvalue%20%3D%20%5B142%2C%2019%5D%22%5D%20%3B%0D%0A131%20-%3E%20132%20%3B%0D%0A133%20%5Blabel%3D%22gini%20%3D%200.413%5Cnsamples%20%3D%2024%5Cnvalue%20%3D%20%5B17%2C%207%5D%22%5D%20%3B%0D%0A132%20-%3E%20133%20%3B%0D%0A134%20%5Blabel%3D%22balance%20%3C%3D%20598.0%5Cngini%20%3D%200.16%5Cnsamples%20%3D%20137%5Cnvalue%20%3D%20%5B125%2C%2012%5D%22%5D%20%3B%0D%0A132%20-%3E%20134%20%3B%0D%0A135%20%5Blabel%3D%22gini%20%3D%200.038%5Cnsamples%20%3D%2051%5Cnvalue%20%3D%20%5B50%2C%201%5D%22%5D%20%3B%0D%0A134%20-%3E%20135%20%3B%0D%0A136%20%5Blabel%3D%22gini%20%3D%200.223%5Cnsamples%20%3D%2086%5Cnvalue%20%3D%20%5B75%2C%2011%5D%22%5D%20%3B%0D%0A134%20-%3E%20136%20%3B%0D%0A137%20%5Blabel%3D%22gini%20%3D%200.484%5Cnsamples%20%3D%2017%5Cnvalue%20%3D%20%5B10%2C%207%5D%22%5D%20%3B%0D%0A131%20-%3E%20137%20%3B%0D%0A138%20%5Blabel%3D%22age%20%3C%3D%2047.5%5Cngini%20%3D%200.489%5Cnsamples%20%3D%2040%5Cnvalue%20%3D%20%5B23%2C%2017%5D%22%5D%20%3B%0D%0A130%20-%3E%20138%20%3B%0D%0A139%20%5Blabel%3D%22gini%20%3D%200.476%5Cnsamples%20%3D%2023%5Cnvalue%20%3D%20%5B9%2C%2014%5D%22%5D%20%3B%0D%0A138%20-%3E%20139%20%3B%0D%0A140%20%5Blabel%3D%22gini%20%3D%200.291%5Cnsamples%20%3D%2017%5Cnvalue%20%3D%20%5B14%2C%203%5D%22%5D%20%3B%0D%0A138%20-%3E%20140%20%3B%0D%0A141%20%5Blabel%3D%22gini%20%3D%200.351%5Cnsamples%20%3D%2022%5Cnvalue%20%3D%20%5B5%2C%2017%5D%22%5D%20%3B%0D%0A127%20-%3E%20141%20%3B%0D%0A142%20%5Blabel%3D%22job_blue-collar%20%3C%3D%200.5%5Cngini%20%3D%200.319%5Cnsamples%20%3D%20236%5Cnvalue%20%3D%20%5B47%2C%20189%5D%22%5D%20%3B%0D%0A78%20-%3E%20142%20%3B%0D%0A143%20%5Blabel%3D%22duration%20%3C%3D%20163.5%5Cngini%20%3D%200.28%5Cnsamples%20%3D%20214%5Cnvalue%20%3D%20%5B36%2C%20178%5D%22%5D%20%3B%0D%0A142%20-%3E%20143%20%3B%0D%0A144%20%5Blabel%3D%22month%20%3C%3D%207.5%5Cngini%20%3D%200.439%5Cnsamples%20%3D%2040%5Cnvalue%20%3D%20%5B13%2C%2027%5D%22%5D%20%3B%0D%0A143%20-%3E%20144%20%3B%0D%0A145%20%5Blabel%3D%22gini%20%3D%200.499%5Cnsamples%20%3D%2023%5Cnvalue%20%3D%20%5B11%2C%2012%5D%22%5D%20%3B%0D%0A144%20-%3E%20145%20%3B%0D%0A146%20%5Blabel%3D%22gini%20%3D%200.208%5Cnsamples%20%3D%2017%5Cnvalue%20%3D%20%5B2%2C%2015%5D%22%5D%20%3B%0D%0A144%20-%3E%20146%20%3B%0D%0A147%20%5Blabel%3D%22age%20%3C%3D%2055.5%5Cngini%20%3D%200.229%5Cnsamples%20%3D%20174%5Cnvalue%20%3D%20%5B23%2C%20151%5D%22%5D%20%3B%0D%0A143%20-%3E%20147%20%3B%0D%0A148%20%5Blabel%3D%22pdays%20%3C%3D%20176.0%5Cngini%20%3D%200.278%5Cnsamples%20%3D%20138%5Cnvalue%20%3D%20%5B23%2C%20115%5D%22%5D%20%3B%0D%0A147%20-%3E%20148%20%3B%0D%0A149%20%5Blabel%3D%22pdays%20%3C%3D%20105.0%5Cngini%20%3D%200.352%5Cnsamples%20%3D%2079%5Cnvalue%20%3D%20%5B18%2C%2061%5D%22%5D%20%3B%0D%0A148%20-%3E%20149%20%3B%0D%0A150%20%5Blabel%3D%22duration%20%3C%3D%20258.5%5Cngini%20%3D%200.27%5Cnsamples%20%3D%2056%5Cnvalue%20%3D%20%5B9%2C%2047%5D%22%5D%20%3B%0D%0A149%20-%3E%20150%20%3B%0D%0A151%20%5Blabel%3D%22day%20%3C%3D%2015.5%5Cngini%20%3D%200.139%5Cnsamples%20%3D%2040%5Cnvalue%20%3D%20%5B3%2C%2037%5D%22%5D%20%3B%0D%0A150%20-%3E%20151%20%3B%0D%0A152%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2027%5Cnvalue%20%3D%20%5B0%2C%2027%5D%22%5D%20%3B%0D%0A151%20-%3E%20152%20%3B%0D%0A153%20%5Blabel%3D%22gini%20%3D%200.355%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B3%2C%2010%5D%22%5D%20%3B%0D%0A151%20-%3E%20153%20%3B%0D%0A154%20%5Blabel%3D%22gini%20%3D%200.469%5Cnsamples%20%3D%2016%5Cnvalue%20%3D%20%5B6%2C%2010%5D%22%5D%20%3B%0D%0A150%20-%3E%20154%20%3B%0D%0A155%20%5Blabel%3D%22gini%20%3D%200.476%5Cnsamples%20%3D%2023%5Cnvalue%20%3D%20%5B9%2C%2014%5D%22%5D%20%3B%0D%0A149%20-%3E%20155%20%3B%0D%0A156%20%5Blabel%3D%22balance%20%3C%3D%201173.0%5Cngini%20%3D%200.155%5Cnsamples%20%3D%2059%5Cnvalue%20%3D%20%5B5%2C%2054%5D%22%5D%20%3B%0D%0A148%20-%3E%20156%20%3B%0D%0A157%20%5Blabel%3D%22gini%20%3D%200.32%5Cnsamples%20%3D%2025%5Cnvalue%20%3D%20%5B5%2C%2020%5D%22%5D%20%3B%0D%0A156%20-%3E%20157%20%3B%0D%0A158%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2034%5Cnvalue%20%3D%20%5B0%2C%2034%5D%22%5D%20%3B%0D%0A156%20-%3E%20158%20%3B%0D%0A159%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2036%5Cnvalue%20%3D%20%5B0%2C%2036%5D%22%5D%20%3B%0D%0A147%20-%3E%20159%20%3B%0D%0A160%20%5Blabel%3D%22gini%20%3D%200.5%5Cnsamples%20%3D%2022%5Cnvalue%20%3D%20%5B11%2C%2011%5D%22%5D%20%3B%0D%0A142%20-%3E%20160%20%3B%0D%0A161%20%5Blabel%3D%22month%20%3C%3D%206.5%5Cngini%20%3D%200.133%5Cnsamples%20%3D%20181%5Cnvalue%20%3D%20%5B168%2C%2013%5D%22%5D%20%3B%0D%0A47%20-%3E%20161%20%3B%0D%0A162%20%5Blabel%3D%22campaign%20%3C%3D%201.5%5Cngini%20%3D%200.06%5Cnsamples%20%3D%20161%5Cnvalue%20%3D%20%5B156%2C%205%5D%22%5D%20%3B%0D%0A161%20-%3E%20162%20%3B%0D%0A163%20%5Blabel%3D%22day%20%3C%3D%2011.5%5Cngini%20%3D%200.153%5Cnsamples%20%3D%2060%5Cnvalue%20%3D%20%5B55%2C%205%5D%22%5D%20%3B%0D%0A162%20-%3E%20163%20%3B%0D%0A164%20%5Blabel%3D%22gini%20%3D%200.32%5Cnsamples%20%3D%2020%5Cnvalue%20%3D%20%5B16%2C%204%5D%22%5D%20%3B%0D%0A163%20-%3E%20164%20%3B%0D%0A165%20%5Blabel%3D%22duration%20%3C%3D%20246.5%5Cngini%20%3D%200.049%5Cnsamples%20%3D%2040%5Cnvalue%20%3D%20%5B39%2C%201%5D%22%5D%20%3B%0D%0A163%20-%3E%20165%20%3B%0D%0A166%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2027%5Cnvalue%20%3D%20%5B27%2C%200%5D%22%5D%20%3B%0D%0A165%20-%3E%20166%20%3B%0D%0A167%20%5Blabel%3D%22gini%20%3D%200.142%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B12%2C%201%5D%22%5D%20%3B%0D%0A165%20-%3E%20167%20%3B%0D%0A168%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%20101%5Cnvalue%20%3D%20%5B101%2C%200%5D%22%5D%20%3B%0D%0A162%20-%3E%20168%20%3B%0D%0A169%20%5Blabel%3D%22gini%20%3D%200.48%5Cnsamples%20%3D%2020%5Cnvalue%20%3D%20%5B12%2C%208%5D%22%5D%20%3B%0D%0A161%20-%3E%20169%20%3B%0D%0A170%20%5Blabel%3D%22month%20%3C%3D%204.5%5Cngini%20%3D%200.215%5Cnsamples%20%3D%202390%5Cnvalue%20%3D%20%5B2097%2C%20293%5D%22%5D%20%3B%0D%0A2%20-%3E%20170%20%3B%0D%0A171%20%5Blabel%3D%22education_tertiary%20%3C%3D%200.5%5Cngini%20%3D%200.416%5Cnsamples%20%3D%20397%5Cnvalue%20%3D%20%5B280%2C%20117%5D%22%5D%20%3B%0D%0A170%20-%3E%20171%20%3B%0D%0A172%20%5Blabel%3D%22duration%20%3C%3D%2086.5%5Cngini%20%3D%200.326%5Cnsamples%20%3D%20258%5Cnvalue%20%3D%20%5B205%2C%2053%5D%22%5D%20%3B%0D%0A171%20-%3E%20172%20%3B%0D%0A173%20%5Blabel%3D%22balance%20%3C%3D%20296.5%5Cngini%20%3D%200.043%5Cnsamples%20%3D%2045%5Cnvalue%20%3D%20%5B44%2C%201%5D%22%5D%20%3B%0D%0A172%20-%3E%20173%20%3B%0D%0A174%20%5Blabel%3D%22gini%20%3D%200.142%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B12%2C%201%5D%22%5D%20%3B%0D%0A173%20-%3E%20174%20%3B%0D%0A175%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2032%5Cnvalue%20%3D%20%5B32%2C%200%5D%22%5D%20%3B%0D%0A173%20-%3E%20175%20%3B%0D%0A176%20%5Blabel%3D%22balance%20%3C%3D%201516.5%5Cngini%20%3D%200.369%5Cnsamples%20%3D%20213%5Cnvalue%20%3D%20%5B161%2C%2052%5D%22%5D%20%3B%0D%0A172%20-%3E%20176%20%3B%0D%0A177%20%5Blabel%3D%22campaign%20%3C%3D%201.5%5Cngini%20%3D%200.327%5Cnsamples%20%3D%20170%5Cnvalue%20%3D%20%5B135%2C%2035%5D%22%5D%20%3B%0D%0A176%20-%3E%20177%20%3B%0D%0A178%20%5Blabel%3D%22job_technician%20%3C%3D%200.5%5Cngini%20%3D%200.399%5Cnsamples%20%3D%2080%5Cnvalue%20%3D%20%5B58%2C%2022%5D%22%5D%20%3B%0D%0A177%20-%3E%20178%20%3B%0D%0A179%20%5Blabel%3D%22job_admin.%20%3C%3D%200.5%5Cngini%20%3D%200.339%5Cnsamples%20%3D%2060%5Cnvalue%20%3D%20%5B47%2C%2013%5D%22%5D%20%3B%0D%0A178%20-%3E%20179%20%3B%0D%0A180%20%5Blabel%3D%22month%20%3C%3D%203.0%5Cngini%20%3D%200.234%5Cnsamples%20%3D%2037%5Cnvalue%20%3D%20%5B32%2C%205%5D%22%5D%20%3B%0D%0A179%20-%3E%20180%20%3B%0D%0A181%20%5Blabel%3D%22gini%20%3D%200.355%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B10%2C%203%5D%22%5D%20%3B%0D%0A180%20-%3E%20181%20%3B%0D%0A182%20%5Blabel%3D%22gini%20%3D%200.153%5Cnsamples%20%3D%2024%5Cnvalue%20%3D%20%5B22%2C%202%5D%22%5D%20%3B%0D%0A180%20-%3E%20182%20%3B%0D%0A183%20%5Blabel%3D%22gini%20%3D%200.454%5Cnsamples%20%3D%2023%5Cnvalue%20%3D%20%5B15%2C%208%5D%22%5D%20%3B%0D%0A179%20-%3E%20183%20%3B%0D%0A184%20%5Blabel%3D%22gini%20%3D%200.495%5Cnsamples%20%3D%2020%5Cnvalue%20%3D%20%5B11%2C%209%5D%22%5D%20%3B%0D%0A178%20-%3E%20184%20%3B%0D%0A185%20%5Blabel%3D%22campaign%20%3C%3D%203.5%5Cngini%20%3D%200.247%5Cnsamples%20%3D%2090%5Cnvalue%20%3D%20%5B77%2C%2013%5D%22%5D%20%3B%0D%0A177%20-%3E%20185%20%3B%0D%0A186%20%5Blabel%3D%22day%20%3C%3D%2021.0%5Cngini%20%3D%200.155%5Cnsamples%20%3D%2071%5Cnvalue%20%3D%20%5B65%2C%206%5D%22%5D%20%3B%0D%0A185%20-%3E%20186%20%3B%0D%0A187%20%5Blabel%3D%22duration%20%3C%3D%20228.5%5Cngini%20%3D%200.07%5Cnsamples%20%3D%2055%5Cnvalue%20%3D%20%5B53%2C%202%5D%22%5D%20%3B%0D%0A186%20-%3E%20187%20%3B%0D%0A188%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2034%5Cnvalue%20%3D%20%5B34%2C%200%5D%22%5D%20%3B%0D%0A187%20-%3E%20188%20%3B%0D%0A189%20%5Blabel%3D%22gini%20%3D%200.172%5Cnsamples%20%3D%2021%5Cnvalue%20%3D%20%5B19%2C%202%5D%22%5D%20%3B%0D%0A187%20-%3E%20189%20%3B%0D%0A190%20%5Blabel%3D%22gini%20%3D%200.375%5Cnsamples%20%3D%2016%5Cnvalue%20%3D%20%5B12%2C%204%5D%22%5D%20%3B%0D%0A186%20-%3E%20190%20%3B%0D%0A191%20%5Blabel%3D%22gini%20%3D%200.465%5Cnsamples%20%3D%2019%5Cnvalue%20%3D%20%5B12%2C%207%5D%22%5D%20%3B%0D%0A185%20-%3E%20191%20%3B%0D%0A192%20%5Blabel%3D%22age%20%3C%3D%2042.5%5Cngini%20%3D%200.478%5Cnsamples%20%3D%2043%5Cnvalue%20%3D%20%5B26%2C%2017%5D%22%5D%20%3B%0D%0A176%20-%3E%20192%20%3B%0D%0A193%20%5Blabel%3D%22gini%20%3D%200.365%5Cnsamples%20%3D%2025%5Cnvalue%20%3D%20%5B19%2C%206%5D%22%5D%20%3B%0D%0A192%20-%3E%20193%20%3B%0D%0A194%20%5Blabel%3D%22gini%20%3D%200.475%5Cnsamples%20%3D%2018%5Cnvalue%20%3D%20%5B7%2C%2011%5D%22%5D%20%3B%0D%0A192%20-%3E%20194%20%3B%0D%0A195%20%5Blabel%3D%22duration%20%3C%3D%20281.5%5Cngini%20%3D%200.497%5Cnsamples%20%3D%20139%5Cnvalue%20%3D%20%5B75%2C%2064%5D%22%5D%20%3B%0D%0A171%20-%3E%20195%20%3B%0D%0A196%20%5Blabel%3D%22duration%20%3C%3D%2060.5%5Cngini%20%3D%200.479%5Cnsamples%20%3D%20118%5Cnvalue%20%3D%20%5B71%2C%2047%5D%22%5D%20%3B%0D%0A195%20-%3E%20196%20%3B%0D%0A197%20%5Blabel%3D%22gini%20%3D%200.142%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B12%2C%201%5D%22%5D%20%3B%0D%0A196%20-%3E%20197%20%3B%0D%0A198%20%5Blabel%3D%22day%20%3C%3D%2028.0%5Cngini%20%3D%200.492%5Cnsamples%20%3D%20105%5Cnvalue%20%3D%20%5B59%2C%2046%5D%22%5D%20%3B%0D%0A196%20-%3E%20198%20%3B%0D%0A199%20%5Blabel%3D%22day%20%3C%3D%2021.0%5Cngini%20%3D%200.499%5Cnsamples%20%3D%2092%5Cnvalue%20%3D%20%5B48%2C%2044%5D%22%5D%20%3B%0D%0A198%20-%3E%20199%20%3B%0D%0A200%20%5Blabel%3D%22marital_married%20%3C%3D%200.5%5Cngini%20%3D%200.486%5Cnsamples%20%3D%2079%5Cnvalue%20%3D%20%5B46%2C%2033%5D%22%5D%20%3B%0D%0A199%20-%3E%20200%20%3B%0D%0A201%20%5Blabel%3D%22gini%20%3D%200.494%5Cnsamples%20%3D%2036%5Cnvalue%20%3D%20%5B16%2C%2020%5D%22%5D%20%3B%0D%0A200%20-%3E%20201%20%3B%0D%0A202%20%5Blabel%3D%22balance%20%3C%3D%201314.0%5Cngini%20%3D%200.422%5Cnsamples%20%3D%2043%5Cnvalue%20%3D%20%5B30%2C%2013%5D%22%5D%20%3B%0D%0A200%20-%3E%20202%20%3B%0D%0A203%20%5Blabel%3D%22gini%20%3D%200.245%5Cnsamples%20%3D%2028%5Cnvalue%20%3D%20%5B24%2C%204%5D%22%5D%20%3B%0D%0A202%20-%3E%20203%20%3B%0D%0A204%20%5Blabel%3D%22gini%20%3D%200.48%5Cnsamples%20%3D%2015%5Cnvalue%20%3D%20%5B6%2C%209%5D%22%5D%20%3B%0D%0A202%20-%3E%20204%20%3B%0D%0A205%20%5Blabel%3D%22gini%20%3D%200.26%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B2%2C%2011%5D%22%5D%20%3B%0D%0A199%20-%3E%20205%20%3B%0D%0A206%20%5Blabel%3D%22gini%20%3D%200.26%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B11%2C%202%5D%22%5D%20%3B%0D%0A198%20-%3E%20206%20%3B%0D%0A207%20%5Blabel%3D%22gini%20%3D%200.308%5Cnsamples%20%3D%2021%5Cnvalue%20%3D%20%5B4%2C%2017%5D%22%5D%20%3B%0D%0A195%20-%3E%20207%20%3B%0D%0A208%20%5Blabel%3D%22pdays%20%3C%3D%20379.0%5Cngini%20%3D%200.161%5Cnsamples%20%3D%201993%5Cnvalue%20%3D%20%5B1817%2C%20176%5D%22%5D%20%3B%0D%0A170%20-%3E%20208%20%3B%0D%0A209%20%5Blabel%3D%22month%20%3C%3D%207.5%5Cngini%20%3D%200.147%5Cnsamples%20%3D%201966%5Cnvalue%20%3D%20%5B1809%2C%20157%5D%22%5D%20%3B%0D%0A208%20-%3E%20209%20%3B%0D%0A210%20%5Blabel%3D%22day%20%3C%3D%204.5%5Cngini%20%3D%200.094%5Cnsamples%20%3D%201579%5Cnvalue%20%3D%20%5B1501%2C%2078%5D%22%5D%20%3B%0D%0A209%20-%3E%20210%20%3B%0D%0A211%20%5Blabel%3D%22contact_unknown%20%3C%3D%200.5%5Cngini%20%3D%200.378%5Cnsamples%20%3D%2099%5Cnvalue%20%3D%20%5B74%2C%2025%5D%22%5D%20%3B%0D%0A210%20-%3E%20211%20%3B%0D%0A212%20%5Blabel%3D%22duration%20%3C%3D%20158.5%5Cngini%20%3D%200.478%5Cnsamples%20%3D%2038%5Cnvalue%20%3D%20%5B15%2C%2023%5D%22%5D%20%3B%0D%0A211%20-%3E%20212%20%3B%0D%0A213%20%5Blabel%3D%22gini%20%3D%200.337%5Cnsamples%20%3D%2014%5Cnvalue%20%3D%20%5B11%2C%203%5D%22%5D%20%3B%0D%0A212%20-%3E%20213%20%3B%0D%0A214%20%5Blabel%3D%22gini%20%3D%200.278%5Cnsamples%20%3D%2024%5Cnvalue%20%3D%20%5B4%2C%2020%5D%22%5D%20%3B%0D%0A212%20-%3E%20214%20%3B%0D%0A215%20%5Blabel%3D%22balance%20%3C%3D%20820.0%5Cngini%20%3D%200.063%5Cnsamples%20%3D%2061%5Cnvalue%20%3D%20%5B59%2C%202%5D%22%5D%20%3B%0D%0A211%20-%3E%20215%20%3B%0D%0A216%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2040%5Cnvalue%20%3D%20%5B40%2C%200%5D%22%5D%20%3B%0D%0A215%20-%3E%20216%20%3B%0D%0A217%20%5Blabel%3D%22gini%20%3D%200.172%5Cnsamples%20%3D%2021%5Cnvalue%20%3D%20%5B19%2C%202%5D%22%5D%20%3B%0D%0A215%20-%3E%20217%20%3B%0D%0A218%20%5Blabel%3D%22contact_unknown%20%3C%3D%200.5%5Cngini%20%3D%200.069%5Cnsamples%20%3D%201480%5Cnvalue%20%3D%20%5B1427%2C%2053%5D%22%5D%20%3B%0D%0A210%20-%3E%20218%20%3B%0D%0A219%20%5Blabel%3D%22duration%20%3C%3D%20173.5%5Cngini%20%3D%200.125%5Cnsamples%20%3D%20730%5Cnvalue%20%3D%20%5B681%2C%2049%5D%22%5D%20%3B%0D%0A218%20-%3E%20219%20%3B%0D%0A220%20%5Blabel%3D%22duration%20%3C%3D%20144.5%5Cngini%20%3D%200.046%5Cnsamples%20%3D%20464%5Cnvalue%20%3D%20%5B453%2C%2011%5D%22%5D%20%3B%0D%0A219%20-%3E%20220%20%3B%0D%0A221%20%5Blabel%3D%22education_tertiary%20%3C%3D%200.5%5Cngini%20%3D%200.026%5Cnsamples%20%3D%20374%5Cnvalue%20%3D%20%5B369%2C%205%5D%22%5D%20%3B%0D%0A220%20-%3E%20221%20%3B%0D%0A222%20%5Blabel%3D%22contact_telephone%20%3C%3D%200.5%5Cngini%20%3D%200.007%5Cnsamples%20%3D%20272%5Cnvalue%20%3D%20%5B271%2C%201%5D%22%5D%20%3B%0D%0A221%20-%3E%20222%20%3B%0D%0A223%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%20234%5Cnvalue%20%3D%20%5B234%2C%200%5D%22%5D%20%3B%0D%0A222%20-%3E%20223%20%3B%0D%0A224%20%5Blabel%3D%22day%20%3C%3D%2012.5%5Cngini%20%3D%200.051%5Cnsamples%20%3D%2038%5Cnvalue%20%3D%20%5B37%2C%201%5D%22%5D%20%3B%0D%0A222%20-%3E%20224%20%3B%0D%0A225%20%5Blabel%3D%22gini%20%3D%200.142%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B12%2C%201%5D%22%5D%20%3B%0D%0A224%20-%3E%20225%20%3B%0D%0A226%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2025%5Cnvalue%20%3D%20%5B25%2C%200%5D%22%5D%20%3B%0D%0A224%20-%3E%20226%20%3B%0D%0A227%20%5Blabel%3D%22age%20%3C%3D%2048.5%5Cngini%20%3D%200.075%5Cnsamples%20%3D%20102%5Cnvalue%20%3D%20%5B98%2C%204%5D%22%5D%20%3B%0D%0A221%20-%3E%20227%20%3B%0D%0A228%20%5Blabel%3D%22duration%20%3C%3D%20114.5%5Cngini%20%3D%200.044%5Cnsamples%20%3D%2089%5Cnvalue%20%3D%20%5B87%2C%202%5D%22%5D%20%3B%0D%0A227%20-%3E%20228%20%3B%0D%0A229%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2066%5Cnvalue%20%3D%20%5B66%2C%200%5D%22%5D%20%3B%0D%0A228%20-%3E%20229%20%3B%0D%0A230%20%5Blabel%3D%22gini%20%3D%200.159%5Cnsamples%20%3D%2023%5Cnvalue%20%3D%20%5B21%2C%202%5D%22%5D%20%3B%0D%0A228%20-%3E%20230%20%3B%0D%0A231%20%5Blabel%3D%22gini%20%3D%200.26%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B11%2C%202%5D%22%5D%20%3B%0D%0A227%20-%3E%20231%20%3B%0D%0A232%20%5Blabel%3D%22age%20%3C%3D%2034.5%5Cngini%20%3D%200.124%5Cnsamples%20%3D%2090%5Cnvalue%20%3D%20%5B84%2C%206%5D%22%5D%20%3B%0D%0A220%20-%3E%20232%20%3B%0D%0A233%20%5Blabel%3D%22age%20%3C%3D%2030.5%5Cngini%20%3D%200.266%5Cnsamples%20%3D%2038%5Cnvalue%20%3D%20%5B32%2C%206%5D%22%5D%20%3B%0D%0A232%20-%3E%20233%20%3B%0D%0A234%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2019%5Cnvalue%20%3D%20%5B19%2C%200%5D%22%5D%20%3B%0D%0A233%20-%3E%20234%20%3B%0D%0A235%20%5Blabel%3D%22gini%20%3D%200.432%5Cnsamples%20%3D%2019%5Cnvalue%20%3D%20%5B13%2C%206%5D%22%5D%20%3B%0D%0A233%20-%3E%20235%20%3B%0D%0A236%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2052%5Cnvalue%20%3D%20%5B52%2C%200%5D%22%5D%20%3B%0D%0A232%20-%3E%20236%20%3B%0D%0A237%20%5Blabel%3D%22day%20%3C%3D%2017.5%5Cngini%20%3D%200.245%5Cnsamples%20%3D%20266%5Cnvalue%20%3D%20%5B228%2C%2038%5D%22%5D%20%3B%0D%0A219%20-%3E%20237%20%3B%0D%0A238%20%5Blabel%3D%22age%20%3C%3D%2037.5%5Cngini%20%3D%200.159%5Cnsamples%20%3D%20184%5Cnvalue%20%3D%20%5B168%2C%2016%5D%22%5D%20%3B%0D%0A237%20-%3E%20238%20%3B%0D%0A239%20%5Blabel%3D%22duration%20%3C%3D%20242.0%5Cngini%20%3D%200.269%5Cnsamples%20%3D%2081%5Cnvalue%20%3D%20%5B68%2C%2013%5D%22%5D%20%3B%0D%0A238%20-%3E%20239%20%3B%0D%0A240%20%5Blabel%3D%22gini%20%3D%200.124%5Cnsamples%20%3D%2030%5Cnvalue%20%3D%20%5B28%2C%202%5D%22%5D%20%3B%0D%0A239%20-%3E%20240%20%3B%0D%0A241%20%5Blabel%3D%22duration%20%3C%3D%20268.0%5Cngini%20%3D%200.338%5Cnsamples%20%3D%2051%5Cnvalue%20%3D%20%5B40%2C%2011%5D%22%5D%20%3B%0D%0A239%20-%3E%20241%20%3B%0D%0A242%20%5Blabel%3D%22gini%20%3D%200.473%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B8%2C%205%5D%22%5D%20%3B%0D%0A241%20-%3E%20242%20%3B%0D%0A243%20%5Blabel%3D%22balance%20%3C%3D%20497.5%5Cngini%20%3D%200.266%5Cnsamples%20%3D%2038%5Cnvalue%20%3D%20%5B32%2C%206%5D%22%5D%20%3B%0D%0A241%20-%3E%20243%20%3B%0D%0A244%20%5Blabel%3D%22gini%20%3D%200.365%5Cnsamples%20%3D%2025%5Cnvalue%20%3D%20%5B19%2C%206%5D%22%5D%20%3B%0D%0A243%20-%3E%20244%20%3B%0D%0A245%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B13%2C%200%5D%22%5D%20%3B%0D%0A243%20-%3E%20245%20%3B%0D%0A246%20%5Blabel%3D%22duration%20%3C%3D%20286.5%5Cngini%20%3D%200.057%5Cnsamples%20%3D%20103%5Cnvalue%20%3D%20%5B100%2C%203%5D%22%5D%20%3B%0D%0A238%20-%3E%20246%20%3B%0D%0A247%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2072%5Cnvalue%20%3D%20%5B72%2C%200%5D%22%5D%20%3B%0D%0A246%20-%3E%20247%20%3B%0D%0A248%20%5Blabel%3D%22gini%20%3D%200.175%5Cnsamples%20%3D%2031%5Cnvalue%20%3D%20%5B28%2C%203%5D%22%5D%20%3B%0D%0A246%20-%3E%20248%20%3B%0D%0A249%20%5Blabel%3D%22pdays%20%3C%3D%204.5%5Cngini%20%3D%200.393%5Cnsamples%20%3D%2082%5Cnvalue%20%3D%20%5B60%2C%2022%5D%22%5D%20%3B%0D%0A237%20-%3E%20249%20%3B%0D%0A250%20%5Blabel%3D%22month%20%3C%3D%206.0%5Cngini%20%3D%200.254%5Cnsamples%20%3D%2067%5Cnvalue%20%3D%20%5B57%2C%2010%5D%22%5D%20%3B%0D%0A249%20-%3E%20250%20%3B%0D%0A251%20%5Blabel%3D%22gini%20%3D%200.48%5Cnsamples%20%3D%2015%5Cnvalue%20%3D%20%5B9%2C%206%5D%22%5D%20%3B%0D%0A250%20-%3E%20251%20%3B%0D%0A252%20%5Blabel%3D%22duration%20%3C%3D%20247.0%5Cngini%20%3D%200.142%5Cnsamples%20%3D%2052%5Cnvalue%20%3D%20%5B48%2C%204%5D%22%5D%20%3B%0D%0A250%20-%3E%20252%20%3B%0D%0A253%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2023%5Cnvalue%20%3D%20%5B23%2C%200%5D%22%5D%20%3B%0D%0A252%20-%3E%20253%20%3B%0D%0A254%20%5Blabel%3D%22gini%20%3D%200.238%5Cnsamples%20%3D%2029%5Cnvalue%20%3D%20%5B25%2C%204%5D%22%5D%20%3B%0D%0A252%20-%3E%20254%20%3B%0D%0A255%20%5Blabel%3D%22gini%20%3D%200.32%5Cnsamples%20%3D%2015%5Cnvalue%20%3D%20%5B3%2C%2012%5D%22%5D%20%3B%0D%0A249%20-%3E%20255%20%3B%0D%0A256%20%5Blabel%3D%22education_unknown%20%3C%3D%200.5%5Cngini%20%3D%200.011%5Cnsamples%20%3D%20750%5Cnvalue%20%3D%20%5B746%2C%204%5D%22%5D%20%3B%0D%0A218%20-%3E%20256%20%3B%0D%0A257%20%5Blabel%3D%22age%20%3C%3D%2026.5%5Cngini%20%3D%200.008%5Cnsamples%20%3D%20720%5Cnvalue%20%3D%20%5B717%2C%203%5D%22%5D%20%3B%0D%0A256%20-%3E%20257%20%3B%0D%0A258%20%5Blabel%3D%22gini%20%3D%200.054%5Cnsamples%20%3D%2036%5Cnvalue%20%3D%20%5B35%2C%201%5D%22%5D%20%3B%0D%0A257%20-%3E%20258%20%3B%0D%0A259%20%5Blabel%3D%22job_admin.%20%3C%3D%200.5%5Cngini%20%3D%200.006%5Cnsamples%20%3D%20684%5Cnvalue%20%3D%20%5B682%2C%202%5D%22%5D%20%3B%0D%0A257%20-%3E%20259%20%3B%0D%0A260%20%5Blabel%3D%22age%20%3C%3D%2031.5%5Cngini%20%3D%200.003%5Cnsamples%20%3D%20602%5Cnvalue%20%3D%20%5B601%2C%201%5D%22%5D%20%3B%0D%0A259%20-%3E%20260%20%3B%0D%0A261%20%5Blabel%3D%22balance%20%3C%3D%203.0%5Cngini%20%3D%200.019%5Cnsamples%20%3D%20102%5Cnvalue%20%3D%20%5B101%2C%201%5D%22%5D%20%3B%0D%0A260%20-%3E%20261%20%3B%0D%0A262%20%5Blabel%3D%22gini%20%3D%200.095%5Cnsamples%20%3D%2020%5Cnvalue%20%3D%20%5B19%2C%201%5D%22%5D%20%3B%0D%0A261%20-%3E%20262%20%3B%0D%0A263%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2082%5Cnvalue%20%3D%20%5B82%2C%200%5D%22%5D%20%3B%0D%0A261%20-%3E%20263%20%3B%0D%0A264%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%20500%5Cnvalue%20%3D%20%5B500%2C%200%5D%22%5D%20%3B%0D%0A260%20-%3E%20264%20%3B%0D%0A265%20%5Blabel%3D%22day%20%3C%3D%206.5%5Cngini%20%3D%200.024%5Cnsamples%20%3D%2082%5Cnvalue%20%3D%20%5B81%2C%201%5D%22%5D%20%3B%0D%0A259%20-%3E%20265%20%3B%0D%0A266%20%5Blabel%3D%22gini%20%3D%200.117%5Cnsamples%20%3D%2016%5Cnvalue%20%3D%20%5B15%2C%201%5D%22%5D%20%3B%0D%0A265%20-%3E%20266%20%3B%0D%0A267%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2066%5Cnvalue%20%3D%20%5B66%2C%200%5D%22%5D%20%3B%0D%0A265%20-%3E%20267%20%3B%0D%0A268%20%5Blabel%3D%22gini%20%3D%200.064%5Cnsamples%20%3D%2030%5Cnvalue%20%3D%20%5B29%2C%201%5D%22%5D%20%3B%0D%0A256%20-%3E%20268%20%3B%0D%0A269%20%5Blabel%3D%22month%20%3C%3D%2010.5%5Cngini%20%3D%200.325%5Cnsamples%20%3D%20387%5Cnvalue%20%3D%20%5B308%2C%2079%5D%22%5D%20%3B%0D%0A209%20-%3E%20269%20%3B%0D%0A270%20%5Blabel%3D%22month%20%3C%3D%208.5%5Cngini%20%3D%200.456%5Cnsamples%20%3D%20176%5Cnvalue%20%3D%20%5B114%2C%2062%5D%22%5D%20%3B%0D%0A269%20-%3E%20270%20%3B%0D%0A271%20%5Blabel%3D%22pdays%20%3C%3D%2087.0%5Cngini%20%3D%200.31%5Cnsamples%20%3D%20120%5Cnvalue%20%3D%20%5B97%2C%2023%5D%22%5D%20%3B%0D%0A270%20-%3E%20271%20%3B%0D%0A272%20%5Blabel%3D%22age%20%3C%3D%2054.5%5Cngini%20%3D%200.178%5Cnsamples%20%3D%20101%5Cnvalue%20%3D%20%5B91%2C%2010%5D%22%5D%20%3B%0D%0A271%20-%3E%20272%20%3B%0D%0A273%20%5Blabel%3D%22marital_single%20%3C%3D%200.5%5Cngini%20%3D%200.107%5Cnsamples%20%3D%2088%5Cnvalue%20%3D%20%5B83%2C%205%5D%22%5D%20%3B%0D%0A272%20-%3E%20273%20%3B%0D%0A274%20%5Blabel%3D%22age%20%3C%3D%2048.5%5Cngini%20%3D%200.028%5Cnsamples%20%3D%2071%5Cnvalue%20%3D%20%5B70%2C%201%5D%22%5D%20%3B%0D%0A273%20-%3E%20274%20%3B%0D%0A275%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2058%5Cnvalue%20%3D%20%5B58%2C%200%5D%22%5D%20%3B%0D%0A274%20-%3E%20275%20%3B%0D%0A276%20%5Blabel%3D%22gini%20%3D%200.142%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B12%2C%201%5D%22%5D%20%3B%0D%0A274%20-%3E%20276%20%3B%0D%0A277%20%5Blabel%3D%22gini%20%3D%200.36%5Cnsamples%20%3D%2017%5Cnvalue%20%3D%20%5B13%2C%204%5D%22%5D%20%3B%0D%0A273%20-%3E%20277%20%3B%0D%0A278%20%5Blabel%3D%22gini%20%3D%200.473%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B8%2C%205%5D%22%5D%20%3B%0D%0A272%20-%3E%20278%20%3B%0D%0A279%20%5Blabel%3D%22gini%20%3D%200.432%5Cnsamples%20%3D%2019%5Cnvalue%20%3D%20%5B6%2C%2013%5D%22%5D%20%3B%0D%0A271%20-%3E%20279%20%3B%0D%0A280%20%5Blabel%3D%22contact_cellular%20%3C%3D%200.5%5Cngini%20%3D%200.423%5Cnsamples%20%3D%2056%5Cnvalue%20%3D%20%5B17%2C%2039%5D%22%5D%20%3B%0D%0A270%20-%3E%20280%20%3B%0D%0A281%20%5Blabel%3D%22gini%20%3D%200.091%5Cnsamples%20%3D%2021%5Cnvalue%20%3D%20%5B1%2C%2020%5D%22%5D%20%3B%0D%0A280%20-%3E%20281%20%3B%0D%0A282%20%5Blabel%3D%22gini%20%3D%200.496%5Cnsamples%20%3D%2035%5Cnvalue%20%3D%20%5B16%2C%2019%5D%22%5D%20%3B%0D%0A280%20-%3E%20282%20%3B%0D%0A283%20%5Blabel%3D%22day%20%3C%3D%2016.5%5Cngini%20%3D%200.148%5Cnsamples%20%3D%20211%5Cnvalue%20%3D%20%5B194%2C%2017%5D%22%5D%20%3B%0D%0A269%20-%3E%20283%20%3B%0D%0A284%20%5Blabel%3D%22gini%20%3D%200.415%5Cnsamples%20%3D%2017%5Cnvalue%20%3D%20%5B5%2C%2012%5D%22%5D%20%3B%0D%0A283%20-%3E%20284%20%3B%0D%0A285%20%5Blabel%3D%22duration%20%3C%3D%20261.5%5Cngini%20%3D%200.05%5Cnsamples%20%3D%20194%5Cnvalue%20%3D%20%5B189%2C%205%5D%22%5D%20%3B%0D%0A283%20-%3E%20285%20%3B%0D%0A286%20%5Blabel%3D%22contact_cellular%20%3C%3D%200.5%5Cngini%20%3D%200.023%5Cnsamples%20%3D%20169%5Cnvalue%20%3D%20%5B167%2C%202%5D%22%5D%20%3B%0D%0A285%20-%3E%20286%20%3B%0D%0A287%20%5Blabel%3D%22gini%20%3D%200.124%5Cnsamples%20%3D%2015%5Cnvalue%20%3D%20%5B14%2C%201%5D%22%5D%20%3B%0D%0A286%20-%3E%20287%20%3B%0D%0A288%20%5Blabel%3D%22age%20%3C%3D%2052.5%5Cngini%20%3D%200.013%5Cnsamples%20%3D%20154%5Cnvalue%20%3D%20%5B153%2C%201%5D%22%5D%20%3B%0D%0A286%20-%3E%20288%20%3B%0D%0A289%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%20132%5Cnvalue%20%3D%20%5B132%2C%200%5D%22%5D%20%3B%0D%0A288%20-%3E%20289%20%3B%0D%0A290%20%5Blabel%3D%22gini%20%3D%200.087%5Cnsamples%20%3D%2022%5Cnvalue%20%3D%20%5B21%2C%201%5D%22%5D%20%3B%0D%0A288%20-%3E%20290%20%3B%0D%0A291%20%5Blabel%3D%22gini%20%3D%200.211%5Cnsamples%20%3D%2025%5Cnvalue%20%3D%20%5B22%2C%203%5D%22%5D%20%3B%0D%0A285%20-%3E%20291%20%3B%0D%0A292%20%5Blabel%3D%22gini%20%3D%200.417%5Cnsamples%20%3D%2027%5Cnvalue%20%3D%20%5B8%2C%2019%5D%22%5D%20%3B%0D%0A208%20-%3E%20292%20%3B%0D%0A293%20%5Blabel%3D%22duration%20%3C%3D%2086.5%5Cngini%20%3D%200.195%5Cnsamples%20%3D%20547%5Cnvalue%20%3D%20%5B60%2C%20487%5D%22%5D%20%3B%0D%0A1%20-%3E%20293%20%3B%0D%0A294%20%5Blabel%3D%22gini%20%3D%200.231%5Cnsamples%20%3D%2015%5Cnvalue%20%3D%20%5B13%2C%202%5D%22%5D%20%3B%0D%0A293%20-%3E%20294%20%3B%0D%0A295%20%5Blabel%3D%22duration%20%3C%3D%20136.5%5Cngini%20%3D%200.161%5Cnsamples%20%3D%20532%5Cnvalue%20%3D%20%5B47%2C%20485%5D%22%5D%20%3B%0D%0A293%20-%3E%20295%20%3B%0D%0A296%20%5Blabel%3D%22pdays%20%3C%3D%2099.5%5Cngini%20%3D%200.406%5Cnsamples%20%3D%2060%5Cnvalue%20%3D%20%5B17%2C%2043%5D%22%5D%20%3B%0D%0A295%20-%3E%20296%20%3B%0D%0A297%20%5Blabel%3D%22gini%20%3D%200.298%5Cnsamples%20%3D%2033%5Cnvalue%20%3D%20%5B6%2C%2027%5D%22%5D%20%3B%0D%0A296%20-%3E%20297%20%3B%0D%0A298%20%5Blabel%3D%22gini%20%3D%200.483%5Cnsamples%20%3D%2027%5Cnvalue%20%3D%20%5B11%2C%2016%5D%22%5D%20%3B%0D%0A296%20-%3E%20298%20%3B%0D%0A299%20%5Blabel%3D%22campaign%20%3C%3D%203.5%5Cngini%20%3D%200.119%5Cnsamples%20%3D%20472%5Cnvalue%20%3D%20%5B30%2C%20442%5D%22%5D%20%3B%0D%0A295%20-%3E%20299%20%3B%0D%0A300%20%5Blabel%3D%22marital_divorced%20%3C%3D%200.5%5Cngini%20%3D%200.109%5Cnsamples%20%3D%20451%5Cnvalue%20%3D%20%5B26%2C%20425%5D%22%5D%20%3B%0D%0A299%20-%3E%20300%20%3B%0D%0A301%20%5Blabel%3D%22duration%20%3C%3D%20195.5%5Cngini%20%3D%200.092%5Cnsamples%20%3D%20414%5Cnvalue%20%3D%20%5B20%2C%20394%5D%22%5D%20%3B%0D%0A300%20-%3E%20301%20%3B%0D%0A302%20%5Blabel%3D%22previous%20%3C%3D%201.5%5Cngini%20%3D%200.167%5Cnsamples%20%3D%20109%5Cnvalue%20%3D%20%5B10%2C%2099%5D%22%5D%20%3B%0D%0A301%20-%3E%20302%20%3B%0D%0A303%20%5Blabel%3D%22duration%20%3C%3D%20184.0%5Cngini%20%3D%200.048%5Cnsamples%20%3D%2041%5Cnvalue%20%3D%20%5B1%2C%2040%5D%22%5D%20%3B%0D%0A302%20-%3E%20303%20%3B%0D%0A304%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2028%5Cnvalue%20%3D%20%5B0%2C%2028%5D%22%5D%20%3B%0D%0A303%20-%3E%20304%20%3B%0D%0A305%20%5Blabel%3D%22gini%20%3D%200.142%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B1%2C%2012%5D%22%5D%20%3B%0D%0A303%20-%3E%20305%20%3B%0D%0A306%20%5Blabel%3D%22job_management%20%3C%3D%200.5%5Cngini%20%3D%200.23%5Cnsamples%20%3D%2068%5Cnvalue%20%3D%20%5B9%2C%2059%5D%22%5D%20%3B%0D%0A302%20-%3E%20306%20%3B%0D%0A307%20%5Blabel%3D%22pdays%20%3C%3D%20131.0%5Cngini%20%3D%200.31%5Cnsamples%20%3D%2047%5Cnvalue%20%3D%20%5B9%2C%2038%5D%22%5D%20%3B%0D%0A306%20-%3E%20307%20%3B%0D%0A308%20%5Blabel%3D%22gini%20%3D%200.383%5Cnsamples%20%3D%2031%5Cnvalue%20%3D%20%5B8%2C%2023%5D%22%5D%20%3B%0D%0A307%20-%3E%20308%20%3B%0D%0A309%20%5Blabel%3D%22gini%20%3D%200.117%5Cnsamples%20%3D%2016%5Cnvalue%20%3D%20%5B1%2C%2015%5D%22%5D%20%3B%0D%0A307%20-%3E%20309%20%3B%0D%0A310%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2021%5Cnvalue%20%3D%20%5B0%2C%2021%5D%22%5D%20%3B%0D%0A306%20-%3E%20310%20%3B%0D%0A311%20%5Blabel%3D%22duration%20%3C%3D%20294.5%5Cngini%20%3D%200.063%5Cnsamples%20%3D%20305%5Cnvalue%20%3D%20%5B10%2C%20295%5D%22%5D%20%3B%0D%0A301%20-%3E%20311%20%3B%0D%0A312%20%5Blabel%3D%22balance%20%3C%3D%20393.5%5Cngini%20%3D%200.029%5Cnsamples%20%3D%20202%5Cnvalue%20%3D%20%5B3%2C%20199%5D%22%5D%20%3B%0D%0A311%20-%3E%20312%20%3B%0D%0A313%20%5Blabel%3D%22balance%20%3C%3D%20282.0%5Cngini%20%3D%200.089%5Cnsamples%20%3D%2064%5Cnvalue%20%3D%20%5B3%2C%2061%5D%22%5D%20%3B%0D%0A312%20-%3E%20313%20%3B%0D%0A314%20%5Blabel%3D%22day%20%3C%3D%2021.5%5Cngini%20%3D%200.038%5Cnsamples%20%3D%2051%5Cnvalue%20%3D%20%5B1%2C%2050%5D%22%5D%20%3B%0D%0A313%20-%3E%20314%20%3B%0D%0A315%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2038%5Cnvalue%20%3D%20%5B0%2C%2038%5D%22%5D%20%3B%0D%0A314%20-%3E%20315%20%3B%0D%0A316%20%5Blabel%3D%22gini%20%3D%200.142%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B1%2C%2012%5D%22%5D%20%3B%0D%0A314%20-%3E%20316%20%3B%0D%0A317%20%5Blabel%3D%22gini%20%3D%200.26%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B2%2C%2011%5D%22%5D%20%3B%0D%0A313%20-%3E%20317%20%3B%0D%0A318%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%20138%5Cnvalue%20%3D%20%5B0%2C%20138%5D%22%5D%20%3B%0D%0A312%20-%3E%20318%20%3B%0D%0A319%20%5Blabel%3D%22duration%20%3C%3D%20303.5%5Cngini%20%3D%200.127%5Cnsamples%20%3D%20103%5Cnvalue%20%3D%20%5B7%2C%2096%5D%22%5D%20%3B%0D%0A311%20-%3E%20319%20%3B%0D%0A320%20%5Blabel%3D%22gini%20%3D%200.337%5Cnsamples%20%3D%2014%5Cnvalue%20%3D%20%5B3%2C%2011%5D%22%5D%20%3B%0D%0A319%20-%3E%20320%20%3B%0D%0A321%20%5Blabel%3D%22pdays%20%3C%3D%20209.0%5Cngini%20%3D%200.086%5Cnsamples%20%3D%2089%5Cnvalue%20%3D%20%5B4%2C%2085%5D%22%5D%20%3B%0D%0A319%20-%3E%20321%20%3B%0D%0A322%20%5Blabel%3D%22marital_single%20%3C%3D%200.5%5Cngini%20%3D%200.053%5Cnsamples%20%3D%2074%5Cnvalue%20%3D%20%5B2%2C%2072%5D%22%5D%20%3B%0D%0A321%20-%3E%20322%20%3B%0D%0A323%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2042%5Cnvalue%20%3D%20%5B0%2C%2042%5D%22%5D%20%3B%0D%0A322%20-%3E%20323%20%3B%0D%0A324%20%5Blabel%3D%22gini%20%3D%200.117%5Cnsamples%20%3D%2032%5Cnvalue%20%3D%20%5B2%2C%2030%5D%22%5D%20%3B%0D%0A322%20-%3E%20324%20%3B%0D%0A325%20%5Blabel%3D%22gini%20%3D%200.231%5Cnsamples%20%3D%2015%5Cnvalue%20%3D%20%5B2%2C%2013%5D%22%5D%20%3B%0D%0A321%20-%3E%20325%20%3B%0D%0A326%20%5Blabel%3D%22balance%20%3C%3D%201502.0%5Cngini%20%3D%200.272%5Cnsamples%20%3D%2037%5Cnvalue%20%3D%20%5B6%2C%2031%5D%22%5D%20%3B%0D%0A300%20-%3E%20326%20%3B%0D%0A327%20%5Blabel%3D%22gini%20%3D%200.087%5Cnsamples%20%3D%2022%5Cnvalue%20%3D%20%5B1%2C%2021%5D%22%5D%20%3B%0D%0A326%20-%3E%20327%20%3B%0D%0A328%20%5Blabel%3D%22gini%20%3D%200.444%5Cnsamples%20%3D%2015%5Cnvalue%20%3D%20%5B5%2C%2010%5D%22%5D%20%3B%0D%0A326%20-%3E%20328%20%3B%0D%0A329%20%5Blabel%3D%22gini%20%3D%200.308%5Cnsamples%20%3D%2021%5Cnvalue%20%3D%20%5B4%2C%2017%5D%22%5D%20%3B%0D%0A299%20-%3E%20329%20%3B%0D%0A330%20%5Blabel%3D%22duration%20%3C%3D%20613.5%5Cngini%20%3D%200.356%5Cnsamples%20%3D%202930%5Cnvalue%20%3D%20%5B680%2C%202250%5D%22%5D%20%3B%0D%0A0%20-%3E%20330%20%5Blabeldistance%3D2.5%2C%20labelangle%3D-45%2C%20headlabel%3D%22False%22%5D%20%3B%0D%0A331%20%5Blabel%3D%22contact_unknown%20%3C%3D%200.5%5Cngini%20%3D%200.447%5Cnsamples%20%3D%201378%5Cnvalue%20%3D%20%5B464%2C%20914%5D%22%5D%20%3B%0D%0A330%20-%3E%20331%20%3B%0D%0A332%20%5Blabel%3D%22poutcome_success%20%3C%3D%200.5%5Cngini%20%3D%200.396%5Cnsamples%20%3D%201136%5Cnvalue%20%3D%20%5B309%2C%20827%5D%22%5D%20%3B%0D%0A331%20-%3E%20332%20%3B%0D%0A333%20%5Blabel%3D%22housing%20%3C%3D%200.5%5Cngini%20%3D%200.427%5Cnsamples%20%3D%20977%5Cnvalue%20%3D%20%5B302%2C%20675%5D%22%5D%20%3B%0D%0A332%20-%3E%20333%20%3B%0D%0A334%20%5Blabel%3D%22loan%20%3C%3D%200.5%5Cngini%20%3D%200.37%5Cnsamples%20%3D%20591%5Cnvalue%20%3D%20%5B145%2C%20446%5D%22%5D%20%3B%0D%0A333%20-%3E%20334%20%3B%0D%0A335%20%5Blabel%3D%22age%20%3C%3D%2060.5%5Cngini%20%3D%200.35%5Cnsamples%20%3D%20549%5Cnvalue%20%3D%20%5B124%2C%20425%5D%22%5D%20%3B%0D%0A334%20-%3E%20335%20%3B%0D%0A336%20%5Blabel%3D%22month%20%3C%3D%202.5%5Cngini%20%3D%200.377%5Cnsamples%20%3D%20468%5Cnvalue%20%3D%20%5B118%2C%20350%5D%22%5D%20%3B%0D%0A335%20-%3E%20336%20%3B%0D%0A337%20%5Blabel%3D%22day%20%3C%3D%2025.5%5Cngini%20%3D%200.492%5Cnsamples%20%3D%2057%5Cnvalue%20%3D%20%5B25%2C%2032%5D%22%5D%20%3B%0D%0A336%20-%3E%20337%20%3B%0D%0A338%20%5Blabel%3D%22day%20%3C%3D%207.5%5Cngini%20%3D%200.42%5Cnsamples%20%3D%2040%5Cnvalue%20%3D%20%5B12%2C%2028%5D%22%5D%20%3B%0D%0A337%20-%3E%20338%20%3B%0D%0A339%20%5Blabel%3D%22gini%20%3D%200.496%5Cnsamples%20%3D%2022%5Cnvalue%20%3D%20%5B10%2C%2012%5D%22%5D%20%3B%0D%0A338%20-%3E%20339%20%3B%0D%0A340%20%5Blabel%3D%22gini%20%3D%200.198%5Cnsamples%20%3D%2018%5Cnvalue%20%3D%20%5B2%2C%2016%5D%22%5D%20%3B%0D%0A338%20-%3E%20340%20%3B%0D%0A341%20%5Blabel%3D%22gini%20%3D%200.36%5Cnsamples%20%3D%2017%5Cnvalue%20%3D%20%5B13%2C%204%5D%22%5D%20%3B%0D%0A337%20-%3E%20341%20%3B%0D%0A342%20%5Blabel%3D%22month%20%3C%3D%206.5%5Cngini%20%3D%200.35%5Cnsamples%20%3D%20411%5Cnvalue%20%3D%20%5B93%2C%20318%5D%22%5D%20%3B%0D%0A336%20-%3E%20342%20%3B%0D%0A343%20%5Blabel%3D%22balance%20%3C%3D%203619.5%5Cngini%20%3D%200.217%5Cnsamples%20%3D%20145%5Cnvalue%20%3D%20%5B18%2C%20127%5D%22%5D%20%3B%0D%0A342%20-%3E%20343%20%3B%0D%0A344%20%5Blabel%3D%22marital_married%20%3C%3D%200.5%5Cngini%20%3D%200.178%5Cnsamples%20%3D%20132%5Cnvalue%20%3D%20%5B13%2C%20119%5D%22%5D%20%3B%0D%0A343%20-%3E%20344%20%3B%0D%0A345%20%5Blabel%3D%22day%20%3C%3D%2015.0%5Cngini%20%3D%200.1%5Cnsamples%20%3D%2076%5Cnvalue%20%3D%20%5B4%2C%2072%5D%22%5D%20%3B%0D%0A344%20-%3E%20345%20%3B%0D%0A346%20%5Blabel%3D%22gini%20%3D%200.213%5Cnsamples%20%3D%2033%5Cnvalue%20%3D%20%5B4%2C%2029%5D%22%5D%20%3B%0D%0A345%20-%3E%20346%20%3B%0D%0A347%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2043%5Cnvalue%20%3D%20%5B0%2C%2043%5D%22%5D%20%3B%0D%0A345%20-%3E%20347%20%3B%0D%0A348%20%5Blabel%3D%22age%20%3C%3D%2046.5%5Cngini%20%3D%200.27%5Cnsamples%20%3D%2056%5Cnvalue%20%3D%20%5B9%2C%2047%5D%22%5D%20%3B%0D%0A344%20-%3E%20348%20%3B%0D%0A349%20%5Blabel%3D%22gini%20%3D%200.384%5Cnsamples%20%3D%2027%5Cnvalue%20%3D%20%5B7%2C%2020%5D%22%5D%20%3B%0D%0A348%20-%3E%20349%20%3B%0D%0A350%20%5Blabel%3D%22gini%20%3D%200.128%5Cnsamples%20%3D%2029%5Cnvalue%20%3D%20%5B2%2C%2027%5D%22%5D%20%3B%0D%0A348%20-%3E%20350%20%3B%0D%0A351%20%5Blabel%3D%22gini%20%3D%200.473%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B5%2C%208%5D%22%5D%20%3B%0D%0A343%20-%3E%20351%20%3B%0D%0A352%20%5Blabel%3D%22pdays%20%3C%3D%204.5%5Cngini%20%3D%200.405%5Cnsamples%20%3D%20266%5Cnvalue%20%3D%20%5B75%2C%20191%5D%22%5D%20%3B%0D%0A342%20-%3E%20352%20%3B%0D%0A353%20%5Blabel%3D%22duration%20%3C%3D%20472.5%5Cngini%20%3D%200.444%5Cnsamples%20%3D%20213%5Cnvalue%20%3D%20%5B71%2C%20142%5D%22%5D%20%3B%0D%0A352%20-%3E%20353%20%3B%0D%0A354%20%5Blabel%3D%22balance%20%3C%3D%20603.0%5Cngini%20%3D%200.493%5Cnsamples%20%3D%20107%5Cnvalue%20%3D%20%5B47%2C%2060%5D%22%5D%20%3B%0D%0A353%20-%3E%20354%20%3B%0D%0A355%20%5Blabel%3D%22duration%20%3C%3D%20417.5%5Cngini%20%3D%200.494%5Cnsamples%20%3D%2054%5Cnvalue%20%3D%20%5B30%2C%2024%5D%22%5D%20%3B%0D%0A354%20-%3E%20355%20%3B%0D%0A356%20%5Blabel%3D%22gini%20%3D%200.49%5Cnsamples%20%3D%2028%5Cnvalue%20%3D%20%5B12%2C%2016%5D%22%5D%20%3B%0D%0A355%20-%3E%20356%20%3B%0D%0A357%20%5Blabel%3D%22gini%20%3D%200.426%5Cnsamples%20%3D%2026%5Cnvalue%20%3D%20%5B18%2C%208%5D%22%5D%20%3B%0D%0A355%20-%3E%20357%20%3B%0D%0A358%20%5Blabel%3D%22duration%20%3C%3D%20426.5%5Cngini%20%3D%200.436%5Cnsamples%20%3D%2053%5Cnvalue%20%3D%20%5B17%2C%2036%5D%22%5D%20%3B%0D%0A354%20-%3E%20358%20%3B%0D%0A359%20%5Blabel%3D%22age%20%3C%3D%2044.5%5Cngini%20%3D%200.488%5Cnsamples%20%3D%2038%5Cnvalue%20%3D%20%5B16%2C%2022%5D%22%5D%20%3B%0D%0A358%20-%3E%20359%20%3B%0D%0A360%20%5Blabel%3D%22gini%20%3D%200.375%5Cnsamples%20%3D%2020%5Cnvalue%20%3D%20%5B5%2C%2015%5D%22%5D%20%3B%0D%0A359%20-%3E%20360%20%3B%0D%0A361%20%5Blabel%3D%22gini%20%3D%200.475%5Cnsamples%20%3D%2018%5Cnvalue%20%3D%20%5B11%2C%207%5D%22%5D%20%3B%0D%0A359%20-%3E%20361%20%3B%0D%0A362%20%5Blabel%3D%22gini%20%3D%200.124%5Cnsamples%20%3D%2015%5Cnvalue%20%3D%20%5B1%2C%2014%5D%22%5D%20%3B%0D%0A358%20-%3E%20362%20%3B%0D%0A363%20%5Blabel%3D%22age%20%3C%3D%2046.5%5Cngini%20%3D%200.35%5Cnsamples%20%3D%20106%5Cnvalue%20%3D%20%5B24%2C%2082%5D%22%5D%20%3B%0D%0A353%20-%3E%20363%20%3B%0D%0A364%20%5Blabel%3D%22age%20%3C%3D%2035.5%5Cngini%20%3D%200.284%5Cnsamples%20%3D%2076%5Cnvalue%20%3D%20%5B13%2C%2063%5D%22%5D%20%3B%0D%0A363%20-%3E%20364%20%3B%0D%0A365%20%5Blabel%3D%22day%20%3C%3D%2016.5%5Cngini%20%3D%200.386%5Cnsamples%20%3D%2046%5Cnvalue%20%3D%20%5B12%2C%2034%5D%22%5D%20%3B%0D%0A364%20-%3E%20365%20%3B%0D%0A366%20%5Blabel%3D%22gini%20%3D%200.18%5Cnsamples%20%3D%2020%5Cnvalue%20%3D%20%5B2%2C%2018%5D%22%5D%20%3B%0D%0A365%20-%3E%20366%20%3B%0D%0A367%20%5Blabel%3D%22gini%20%3D%200.473%5Cnsamples%20%3D%2026%5Cnvalue%20%3D%20%5B10%2C%2016%5D%22%5D%20%3B%0D%0A365%20-%3E%20367%20%3B%0D%0A368%20%5Blabel%3D%22gini%20%3D%200.064%5Cnsamples%20%3D%2030%5Cnvalue%20%3D%20%5B1%2C%2029%5D%22%5D%20%3B%0D%0A364%20-%3E%20368%20%3B%0D%0A369%20%5Blabel%3D%22gini%20%3D%200.464%5Cnsamples%20%3D%2030%5Cnvalue%20%3D%20%5B11%2C%2019%5D%22%5D%20%3B%0D%0A363%20-%3E%20369%20%3B%0D%0A370%20%5Blabel%3D%22previous%20%3C%3D%203.5%5Cngini%20%3D%200.14%5Cnsamples%20%3D%2053%5Cnvalue%20%3D%20%5B4%2C%2049%5D%22%5D%20%3B%0D%0A352%20-%3E%20370%20%3B%0D%0A371%20%5Blabel%3D%22day%20%3C%3D%2011.5%5Cngini%20%3D%200.051%5Cnsamples%20%3D%2038%5Cnvalue%20%3D%20%5B1%2C%2037%5D%22%5D%20%3B%0D%0A370%20-%3E%20371%20%3B%0D%0A372%20%5Blabel%3D%22gini%20%3D%200.142%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B1%2C%2012%5D%22%5D%20%3B%0D%0A371%20-%3E%20372%20%3B%0D%0A373%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2025%5Cnvalue%20%3D%20%5B0%2C%2025%5D%22%5D%20%3B%0D%0A371%20-%3E%20373%20%3B%0D%0A374%20%5Blabel%3D%22gini%20%3D%200.32%5Cnsamples%20%3D%2015%5Cnvalue%20%3D%20%5B3%2C%2012%5D%22%5D%20%3B%0D%0A370%20-%3E%20374%20%3B%0D%0A375%20%5Blabel%3D%22balance%20%3C%3D%201834.0%5Cngini%20%3D%200.137%5Cnsamples%20%3D%2081%5Cnvalue%20%3D%20%5B6%2C%2075%5D%22%5D%20%3B%0D%0A335%20-%3E%20375%20%3B%0D%0A376%20%5Blabel%3D%22age%20%3C%3D%2074.5%5Cngini%20%3D%200.033%5Cnsamples%20%3D%2059%5Cnvalue%20%3D%20%5B1%2C%2058%5D%22%5D%20%3B%0D%0A375%20-%3E%20376%20%3B%0D%0A377%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2045%5Cnvalue%20%3D%20%5B0%2C%2045%5D%22%5D%20%3B%0D%0A376%20-%3E%20377%20%3B%0D%0A378%20%5Blabel%3D%22gini%20%3D%200.133%5Cnsamples%20%3D%2014%5Cnvalue%20%3D%20%5B1%2C%2013%5D%22%5D%20%3B%0D%0A376%20-%3E%20378%20%3B%0D%0A379%20%5Blabel%3D%22gini%20%3D%200.351%5Cnsamples%20%3D%2022%5Cnvalue%20%3D%20%5B5%2C%2017%5D%22%5D%20%3B%0D%0A375%20-%3E%20379%20%3B%0D%0A380%20%5Blabel%3D%22day%20%3C%3D%2018.0%5Cngini%20%3D%200.5%5Cnsamples%20%3D%2042%5Cnvalue%20%3D%20%5B21%2C%2021%5D%22%5D%20%3B%0D%0A334%20-%3E%20380%20%3B%0D%0A381%20%5Blabel%3D%22gini%20%3D%200.473%5Cnsamples%20%3D%2026%5Cnvalue%20%3D%20%5B16%2C%2010%5D%22%5D%20%3B%0D%0A380%20-%3E%20381%20%3B%0D%0A382%20%5Blabel%3D%22gini%20%3D%200.43%5Cnsamples%20%3D%2016%5Cnvalue%20%3D%20%5B5%2C%2011%5D%22%5D%20%3B%0D%0A380%20-%3E%20382%20%3B%0D%0A383%20%5Blabel%3D%22pdays%20%3C%3D%20369.5%5Cngini%20%3D%200.483%5Cnsamples%20%3D%20386%5Cnvalue%20%3D%20%5B157%2C%20229%5D%22%5D%20%3B%0D%0A333%20-%3E%20383%20%3B%0D%0A384%20%5Blabel%3D%22duration%20%3C%3D%20410.0%5Cngini%20%3D%200.49%5Cnsamples%20%3D%20367%5Cnvalue%20%3D%20%5B157%2C%20210%5D%22%5D%20%3B%0D%0A383%20-%3E%20384%20%3B%0D%0A385%20%5Blabel%3D%22balance%20%3C%3D%205.5%5Cngini%20%3D%200.491%5Cnsamples%20%3D%2090%5Cnvalue%20%3D%20%5B51%2C%2039%5D%22%5D%20%3B%0D%0A384%20-%3E%20385%20%3B%0D%0A386%20%5Blabel%3D%22gini%20%3D%200.459%5Cnsamples%20%3D%2014%5Cnvalue%20%3D%20%5B5%2C%209%5D%22%5D%20%3B%0D%0A385%20-%3E%20386%20%3B%0D%0A387%20%5Blabel%3D%22balance%20%3C%3D%20423.0%5Cngini%20%3D%200.478%5Cnsamples%20%3D%2076%5Cnvalue%20%3D%20%5B46%2C%2030%5D%22%5D%20%3B%0D%0A385%20-%3E%20387%20%3B%0D%0A388%20%5Blabel%3D%22gini%20%3D%200.375%5Cnsamples%20%3D%2024%5Cnvalue%20%3D%20%5B18%2C%206%5D%22%5D%20%3B%0D%0A387%20-%3E%20388%20%3B%0D%0A389%20%5Blabel%3D%22duration%20%3C%3D%20397.5%5Cngini%20%3D%200.497%5Cnsamples%20%3D%2052%5Cnvalue%20%3D%20%5B28%2C%2024%5D%22%5D%20%3B%0D%0A387%20-%3E%20389%20%3B%0D%0A390%20%5Blabel%3D%22duration%20%3C%3D%20375.0%5Cngini%20%3D%200.499%5Cnsamples%20%3D%2038%5Cnvalue%20%3D%20%5B18%2C%2020%5D%22%5D%20%3B%0D%0A389%20-%3E%20390%20%3B%0D%0A391%20%5Blabel%3D%22gini%20%3D%200.426%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B9%2C%204%5D%22%5D%20%3B%0D%0A390%20-%3E%20391%20%3B%0D%0A392%20%5Blabel%3D%22gini%20%3D%200.461%5Cnsamples%20%3D%2025%5Cnvalue%20%3D%20%5B9%2C%2016%5D%22%5D%20%3B%0D%0A390%20-%3E%20392%20%3B%0D%0A393%20%5Blabel%3D%22gini%20%3D%200.408%5Cnsamples%20%3D%2014%5Cnvalue%20%3D%20%5B10%2C%204%5D%22%5D%20%3B%0D%0A389%20-%3E%20393%20%3B%0D%0A394%20%5Blabel%3D%22age%20%3C%3D%2030.5%5Cngini%20%3D%200.472%5Cnsamples%20%3D%20277%5Cnvalue%20%3D%20%5B106%2C%20171%5D%22%5D%20%3B%0D%0A384%20-%3E%20394%20%3B%0D%0A395%20%5Blabel%3D%22campaign%20%3C%3D%202.5%5Cngini%20%3D%200.325%5Cnsamples%20%3D%2049%5Cnvalue%20%3D%20%5B10%2C%2039%5D%22%5D%20%3B%0D%0A394%20-%3E%20395%20%3B%0D%0A396%20%5Blabel%3D%22gini%20%3D%200.165%5Cnsamples%20%3D%2033%5Cnvalue%20%3D%20%5B3%2C%2030%5D%22%5D%20%3B%0D%0A395%20-%3E%20396%20%3B%0D%0A397%20%5Blabel%3D%22gini%20%3D%200.492%5Cnsamples%20%3D%2016%5Cnvalue%20%3D%20%5B7%2C%209%5D%22%5D%20%3B%0D%0A395%20-%3E%20397%20%3B%0D%0A398%20%5Blabel%3D%22age%20%3C%3D%2056.5%5Cngini%20%3D%200.488%5Cnsamples%20%3D%20228%5Cnvalue%20%3D%20%5B96%2C%20132%5D%22%5D%20%3B%0D%0A394%20-%3E%20398%20%3B%0D%0A399%20%5Blabel%3D%22duration%20%3C%3D%20434.5%5Cngini%20%3D%200.494%5Cnsamples%20%3D%20208%5Cnvalue%20%3D%20%5B93%2C%20115%5D%22%5D%20%3B%0D%0A398%20-%3E%20399%20%3B%0D%0A400%20%5Blabel%3D%22gini%20%3D%200.42%5Cnsamples%20%3D%2020%5Cnvalue%20%3D%20%5B14%2C%206%5D%22%5D%20%3B%0D%0A399%20-%3E%20400%20%3B%0D%0A401%20%5Blabel%3D%22balance%20%3C%3D%2043.5%5Cngini%20%3D%200.487%5Cnsamples%20%3D%20188%5Cnvalue%20%3D%20%5B79%2C%20109%5D%22%5D%20%3B%0D%0A399%20-%3E%20401%20%3B%0D%0A402%20%5Blabel%3D%22day%20%3C%3D%2012.5%5Cngini%20%3D%200.482%5Cnsamples%20%3D%2037%5Cnvalue%20%3D%20%5B22%2C%2015%5D%22%5D%20%3B%0D%0A401%20-%3E%20402%20%3B%0D%0A403%20%5Blabel%3D%22gini%20%3D%200.391%5Cnsamples%20%3D%2015%5Cnvalue%20%3D%20%5B11%2C%204%5D%22%5D%20%3B%0D%0A402%20-%3E%20403%20%3B%0D%0A404%20%5Blabel%3D%22gini%20%3D%200.5%5Cnsamples%20%3D%2022%5Cnvalue%20%3D%20%5B11%2C%2011%5D%22%5D%20%3B%0D%0A402%20-%3E%20404%20%3B%0D%0A405%20%5Blabel%3D%22age%20%3C%3D%2039.5%5Cngini%20%3D%200.47%5Cnsamples%20%3D%20151%5Cnvalue%20%3D%20%5B57%2C%2094%5D%22%5D%20%3B%0D%0A401%20-%3E%20405%20%3B%0D%0A406%20%5Blabel%3D%22balance%20%3C%3D%202150.5%5Cngini%20%3D%200.44%5Cnsamples%20%3D%2095%5Cnvalue%20%3D%20%5B31%2C%2064%5D%22%5D%20%3B%0D%0A405%20-%3E%20406%20%3B%0D%0A407%20%5Blabel%3D%22day%20%3C%3D%2013.5%5Cngini%20%3D%200.394%5Cnsamples%20%3D%2074%5Cnvalue%20%3D%20%5B20%2C%2054%5D%22%5D%20%3B%0D%0A406%20-%3E%20407%20%3B%0D%0A408%20%5Blabel%3D%22gini%20%3D%200.211%5Cnsamples%20%3D%2025%5Cnvalue%20%3D%20%5B3%2C%2022%5D%22%5D%20%3B%0D%0A407%20-%3E%20408%20%3B%0D%0A409%20%5Blabel%3D%22marital_married%20%3C%3D%200.5%5Cngini%20%3D%200.453%5Cnsamples%20%3D%2049%5Cnvalue%20%3D%20%5B17%2C%2032%5D%22%5D%20%3B%0D%0A407%20-%3E%20409%20%3B%0D%0A410%20%5Blabel%3D%22gini%20%3D%200.32%5Cnsamples%20%3D%2020%5Cnvalue%20%3D%20%5B4%2C%2016%5D%22%5D%20%3B%0D%0A409%20-%3E%20410%20%3B%0D%0A411%20%5Blabel%3D%22gini%20%3D%200.495%5Cnsamples%20%3D%2029%5Cnvalue%20%3D%20%5B13%2C%2016%5D%22%5D%20%3B%0D%0A409%20-%3E%20411%20%3B%0D%0A412%20%5Blabel%3D%22gini%20%3D%200.499%5Cnsamples%20%3D%2021%5Cnvalue%20%3D%20%5B11%2C%2010%5D%22%5D%20%3B%0D%0A406%20-%3E%20412%20%3B%0D%0A413%20%5Blabel%3D%22balance%20%3C%3D%201554.0%5Cngini%20%3D%200.497%5Cnsamples%20%3D%2056%5Cnvalue%20%3D%20%5B26%2C%2030%5D%22%5D%20%3B%0D%0A405%20-%3E%20413%20%3B%0D%0A414%20%5Blabel%3D%22gini%20%3D%200.475%5Cnsamples%20%3D%2036%5Cnvalue%20%3D%20%5B22%2C%2014%5D%22%5D%20%3B%0D%0A413%20-%3E%20414%20%3B%0D%0A415%20%5Blabel%3D%22gini%20%3D%200.32%5Cnsamples%20%3D%2020%5Cnvalue%20%3D%20%5B4%2C%2016%5D%22%5D%20%3B%0D%0A413%20-%3E%20415%20%3B%0D%0A416%20%5Blabel%3D%22gini%20%3D%200.255%5Cnsamples%20%3D%2020%5Cnvalue%20%3D%20%5B3%2C%2017%5D%22%5D%20%3B%0D%0A398%20-%3E%20416%20%3B%0D%0A417%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2019%5Cnvalue%20%3D%20%5B0%2C%2019%5D%22%5D%20%3B%0D%0A383%20-%3E%20417%20%3B%0D%0A418%20%5Blabel%3D%22pdays%20%3C%3D%20186.5%5Cngini%20%3D%200.084%5Cnsamples%20%3D%20159%5Cnvalue%20%3D%20%5B7%2C%20152%5D%22%5D%20%3B%0D%0A332%20-%3E%20418%20%3B%0D%0A419%20%5Blabel%3D%22month%20%3C%3D%209.5%5Cngini%20%3D%200.033%5Cnsamples%20%3D%20118%5Cnvalue%20%3D%20%5B2%2C%20116%5D%22%5D%20%3B%0D%0A418%20-%3E%20419%20%3B%0D%0A420%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2095%5Cnvalue%20%3D%20%5B0%2C%2095%5D%22%5D%20%3B%0D%0A419%20-%3E%20420%20%3B%0D%0A421%20%5Blabel%3D%22gini%20%3D%200.159%5Cnsamples%20%3D%2023%5Cnvalue%20%3D%20%5B2%2C%2021%5D%22%5D%20%3B%0D%0A419%20-%3E%20421%20%3B%0D%0A422%20%5Blabel%3D%22balance%20%3C%3D%20731.5%5Cngini%20%3D%200.214%5Cnsamples%20%3D%2041%5Cnvalue%20%3D%20%5B5%2C%2036%5D%22%5D%20%3B%0D%0A418%20-%3E%20422%20%3B%0D%0A423%20%5Blabel%3D%22gini%20%3D%200.32%5Cnsamples%20%3D%2020%5Cnvalue%20%3D%20%5B4%2C%2016%5D%22%5D%20%3B%0D%0A422%20-%3E%20423%20%3B%0D%0A424%20%5Blabel%3D%22gini%20%3D%200.091%5Cnsamples%20%3D%2021%5Cnvalue%20%3D%20%5B1%2C%2020%5D%22%5D%20%3B%0D%0A422%20-%3E%20424%20%3B%0D%0A425%20%5Blabel%3D%22duration%20%3C%3D%20472.5%5Cngini%20%3D%200.461%5Cnsamples%20%3D%20242%5Cnvalue%20%3D%20%5B155%2C%2087%5D%22%5D%20%3B%0D%0A331%20-%3E%20425%20%3B%0D%0A426%20%5Blabel%3D%22age%20%3C%3D%2030.5%5Cngini%20%3D%200.295%5Cnsamples%20%3D%20117%5Cnvalue%20%3D%20%5B96%2C%2021%5D%22%5D%20%3B%0D%0A425%20-%3E%20426%20%3B%0D%0A427%20%5Blabel%3D%22gini%20%3D%200.483%5Cnsamples%20%3D%2027%5Cnvalue%20%3D%20%5B16%2C%2011%5D%22%5D%20%3B%0D%0A426%20-%3E%20427%20%3B%0D%0A428%20%5Blabel%3D%22job_management%20%3C%3D%200.5%5Cngini%20%3D%200.198%5Cnsamples%20%3D%2090%5Cnvalue%20%3D%20%5B80%2C%2010%5D%22%5D%20%3B%0D%0A426%20-%3E%20428%20%3B%0D%0A429%20%5Blabel%3D%22duration%20%3C%3D%20441.5%5Cngini%20%3D%200.126%5Cnsamples%20%3D%2074%5Cnvalue%20%3D%20%5B69%2C%205%5D%22%5D%20%3B%0D%0A428%20-%3E%20429%20%3B%0D%0A430%20%5Blabel%3D%22month%20%3C%3D%205.5%5Cngini%20%3D%200.067%5Cnsamples%20%3D%2058%5Cnvalue%20%3D%20%5B56%2C%202%5D%22%5D%20%3B%0D%0A429%20-%3E%20430%20%3B%0D%0A431%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2033%5Cnvalue%20%3D%20%5B33%2C%200%5D%22%5D%20%3B%0D%0A430%20-%3E%20431%20%3B%0D%0A432%20%5Blabel%3D%22gini%20%3D%200.147%5Cnsamples%20%3D%2025%5Cnvalue%20%3D%20%5B23%2C%202%5D%22%5D%20%3B%0D%0A430%20-%3E%20432%20%3B%0D%0A433%20%5Blabel%3D%22gini%20%3D%200.305%5Cnsamples%20%3D%2016%5Cnvalue%20%3D%20%5B13%2C%203%5D%22%5D%20%3B%0D%0A429%20-%3E%20433%20%3B%0D%0A434%20%5Blabel%3D%22gini%20%3D%200.43%5Cnsamples%20%3D%2016%5Cnvalue%20%3D%20%5B11%2C%205%5D%22%5D%20%3B%0D%0A428%20-%3E%20434%20%3B%0D%0A435%20%5Blabel%3D%22month%20%3C%3D%205.5%5Cngini%20%3D%200.498%5Cnsamples%20%3D%20125%5Cnvalue%20%3D%20%5B59%2C%2066%5D%22%5D%20%3B%0D%0A425%20-%3E%20435%20%3B%0D%0A436%20%5Blabel%3D%22age%20%3C%3D%2030.5%5Cngini%20%3D%200.493%5Cnsamples%20%3D%2075%5Cnvalue%20%3D%20%5B42%2C%2033%5D%22%5D%20%3B%0D%0A435%20-%3E%20436%20%3B%0D%0A437%20%5Blabel%3D%22gini%20%3D%200.245%5Cnsamples%20%3D%2014%5Cnvalue%20%3D%20%5B12%2C%202%5D%22%5D%20%3B%0D%0A436%20-%3E%20437%20%3B%0D%0A438%20%5Blabel%3D%22age%20%3C%3D%2050.0%5Cngini%20%3D%200.5%5Cnsamples%20%3D%2061%5Cnvalue%20%3D%20%5B30%2C%2031%5D%22%5D%20%3B%0D%0A436%20-%3E%20438%20%3B%0D%0A439%20%5Blabel%3D%22duration%20%3C%3D%20556.0%5Cngini%20%3D%200.489%5Cnsamples%20%3D%2047%5Cnvalue%20%3D%20%5B20%2C%2027%5D%22%5D%20%3B%0D%0A438%20-%3E%20439%20%3B%0D%0A440%20%5Blabel%3D%22gini%20%3D%200.417%5Cnsamples%20%3D%2027%5Cnvalue%20%3D%20%5B8%2C%2019%5D%22%5D%20%3B%0D%0A439%20-%3E%20440%20%3B%0D%0A441%20%5Blabel%3D%22gini%20%3D%200.48%5Cnsamples%20%3D%2020%5Cnvalue%20%3D%20%5B12%2C%208%5D%22%5D%20%3B%0D%0A439%20-%3E%20441%20%3B%0D%0A442%20%5Blabel%3D%22gini%20%3D%200.408%5Cnsamples%20%3D%2014%5Cnvalue%20%3D%20%5B10%2C%204%5D%22%5D%20%3B%0D%0A438%20-%3E%20442%20%3B%0D%0A443%20%5Blabel%3D%22housing%20%3C%3D%200.5%5Cngini%20%3D%200.449%5Cnsamples%20%3D%2050%5Cnvalue%20%3D%20%5B17%2C%2033%5D%22%5D%20%3B%0D%0A435%20-%3E%20443%20%3B%0D%0A444%20%5Blabel%3D%22gini%20%3D%200.497%5Cnsamples%20%3D%2024%5Cnvalue%20%3D%20%5B13%2C%2011%5D%22%5D%20%3B%0D%0A443%20-%3E%20444%20%3B%0D%0A445%20%5Blabel%3D%22gini%20%3D%200.26%5Cnsamples%20%3D%2026%5Cnvalue%20%3D%20%5B4%2C%2022%5D%22%5D%20%3B%0D%0A443%20-%3E%20445%20%3B%0D%0A446%20%5Blabel%3D%22duration%20%3C%3D%20794.5%5Cngini%20%3D%200.24%5Cnsamples%20%3D%201552%5Cnvalue%20%3D%20%5B216%2C%201336%5D%22%5D%20%3B%0D%0A330%20-%3E%20446%20%3B%0D%0A447%20%5Blabel%3D%22contact_unknown%20%3C%3D%200.5%5Cngini%20%3D%200.322%5Cnsamples%20%3D%20615%5Cnvalue%20%3D%20%5B124%2C%20491%5D%22%5D%20%3B%0D%0A446%20-%3E%20447%20%3B%0D%0A448%20%5Blabel%3D%22poutcome_success%20%3C%3D%200.5%5Cngini%20%3D%200.291%5Cnsamples%20%3D%20504%5Cnvalue%20%3D%20%5B89%2C%20415%5D%22%5D%20%3B%0D%0A447%20-%3E%20448%20%3B%0D%0A449%20%5Blabel%3D%22duration%20%3C%3D%20787.5%5Cngini%20%3D%200.311%5Cnsamples%20%3D%20457%5Cnvalue%20%3D%20%5B88%2C%20369%5D%22%5D%20%3B%0D%0A448%20-%3E%20449%20%3B%0D%0A450%20%5Blabel%3D%22day%20%3C%3D%2013.5%5Cngini%20%3D%200.301%5Cnsamples%20%3D%20444%5Cnvalue%20%3D%20%5B82%2C%20362%5D%22%5D%20%3B%0D%0A449%20-%3E%20450%20%3B%0D%0A451%20%5Blabel%3D%22day%20%3C%3D%203.5%5Cngini%20%3D%200.216%5Cnsamples%20%3D%20179%5Cnvalue%20%3D%20%5B22%2C%20157%5D%22%5D%20%3B%0D%0A450%20-%3E%20451%20%3B%0D%0A452%20%5Blabel%3D%22gini%20%3D%200.417%5Cnsamples%20%3D%2027%5Cnvalue%20%3D%20%5B8%2C%2019%5D%22%5D%20%3B%0D%0A451%20-%3E%20452%20%3B%0D%0A453%20%5Blabel%3D%22day%20%3C%3D%2011.5%5Cngini%20%3D%200.167%5Cnsamples%20%3D%20152%5Cnvalue%20%3D%20%5B14%2C%20138%5D%22%5D%20%3B%0D%0A451%20-%3E%20453%20%3B%0D%0A454%20%5Blabel%3D%22day%20%3C%3D%2010.5%5Cngini%20%3D%200.209%5Cnsamples%20%3D%20118%5Cnvalue%20%3D%20%5B14%2C%20104%5D%22%5D%20%3B%0D%0A453%20-%3E%20454%20%3B%0D%0A455%20%5Blabel%3D%22duration%20%3C%3D%20715.5%5Cngini%20%3D%200.149%5Cnsamples%20%3D%2099%5Cnvalue%20%3D%20%5B8%2C%2091%5D%22%5D%20%3B%0D%0A454%20-%3E%20455%20%3B%0D%0A456%20%5Blabel%3D%22day%20%3C%3D%205.5%5Cngini%20%3D%200.234%5Cnsamples%20%3D%2059%5Cnvalue%20%3D%20%5B8%2C%2051%5D%22%5D%20%3B%0D%0A455%20-%3E%20456%20%3B%0D%0A457%20%5Blabel%3D%22gini%20%3D%200.401%5Cnsamples%20%3D%2018%5Cnvalue%20%3D%20%5B5%2C%2013%5D%22%5D%20%3B%0D%0A456%20-%3E%20457%20%3B%0D%0A458%20%5Blabel%3D%22duration%20%3C%3D%20666.5%5Cngini%20%3D%200.136%5Cnsamples%20%3D%2041%5Cnvalue%20%3D%20%5B3%2C%2038%5D%22%5D%20%3B%0D%0A456%20-%3E%20458%20%3B%0D%0A459%20%5Blabel%3D%22gini%20%3D%200.211%5Cnsamples%20%3D%2025%5Cnvalue%20%3D%20%5B3%2C%2022%5D%22%5D%20%3B%0D%0A458%20-%3E%20459%20%3B%0D%0A460%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2016%5Cnvalue%20%3D%20%5B0%2C%2016%5D%22%5D%20%3B%0D%0A458%20-%3E%20460%20%3B%0D%0A461%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2040%5Cnvalue%20%3D%20%5B0%2C%2040%5D%22%5D%20%3B%0D%0A455%20-%3E%20461%20%3B%0D%0A462%20%5Blabel%3D%22gini%20%3D%200.432%5Cnsamples%20%3D%2019%5Cnvalue%20%3D%20%5B6%2C%2013%5D%22%5D%20%3B%0D%0A454%20-%3E%20462%20%3B%0D%0A463%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2034%5Cnvalue%20%3D%20%5B0%2C%2034%5D%22%5D%20%3B%0D%0A453%20-%3E%20463%20%3B%0D%0A464%20%5Blabel%3D%22day%20%3C%3D%2020.5%5Cngini%20%3D%200.35%5Cnsamples%20%3D%20265%5Cnvalue%20%3D%20%5B60%2C%20205%5D%22%5D%20%3B%0D%0A450%20-%3E%20464%20%3B%0D%0A465%20%5Blabel%3D%22month%20%3C%3D%204.5%5Cngini%20%3D%200.417%5Cnsamples%20%3D%20145%5Cnvalue%20%3D%20%5B43%2C%20102%5D%22%5D%20%3B%0D%0A464%20-%3E%20465%20%3B%0D%0A466%20%5Blabel%3D%22gini%20%3D%200.497%5Cnsamples%20%3D%2024%5Cnvalue%20%3D%20%5B13%2C%2011%5D%22%5D%20%3B%0D%0A465%20-%3E%20466%20%3B%0D%0A467%20%5Blabel%3D%22month%20%3C%3D%206.5%5Cngini%20%3D%200.373%5Cnsamples%20%3D%20121%5Cnvalue%20%3D%20%5B30%2C%2091%5D%22%5D%20%3B%0D%0A465%20-%3E%20467%20%3B%0D%0A468%20%5Blabel%3D%22duration%20%3C%3D%20683.0%5Cngini%20%3D%200.193%5Cnsamples%20%3D%2037%5Cnvalue%20%3D%20%5B4%2C%2033%5D%22%5D%20%3B%0D%0A467%20-%3E%20468%20%3B%0D%0A469%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2020%5Cnvalue%20%3D%20%5B0%2C%2020%5D%22%5D%20%3B%0D%0A468%20-%3E%20469%20%3B%0D%0A470%20%5Blabel%3D%22gini%20%3D%200.36%5Cnsamples%20%3D%2017%5Cnvalue%20%3D%20%5B4%2C%2013%5D%22%5D%20%3B%0D%0A468%20-%3E%20470%20%3B%0D%0A471%20%5Blabel%3D%22duration%20%3C%3D%20673.5%5Cngini%20%3D%200.427%5Cnsamples%20%3D%2084%5Cnvalue%20%3D%20%5B26%2C%2058%5D%22%5D%20%3B%0D%0A467%20-%3E%20471%20%3B%0D%0A472%20%5Blabel%3D%22month%20%3C%3D%209.0%5Cngini%20%3D%200.492%5Cnsamples%20%3D%2039%5Cnvalue%20%3D%20%5B17%2C%2022%5D%22%5D%20%3B%0D%0A471%20-%3E%20472%20%3B%0D%0A473%20%5Blabel%3D%22gini%20%3D%200.408%5Cnsamples%20%3D%2021%5Cnvalue%20%3D%20%5B6%2C%2015%5D%22%5D%20%3B%0D%0A472%20-%3E%20473%20%3B%0D%0A474%20%5Blabel%3D%22gini%20%3D%200.475%5Cnsamples%20%3D%2018%5Cnvalue%20%3D%20%5B11%2C%207%5D%22%5D%20%3B%0D%0A472%20-%3E%20474%20%3B%0D%0A475%20%5Blabel%3D%22duration%20%3C%3D%20734.0%5Cngini%20%3D%200.32%5Cnsamples%20%3D%2045%5Cnvalue%20%3D%20%5B9%2C%2036%5D%22%5D%20%3B%0D%0A471%20-%3E%20475%20%3B%0D%0A476%20%5Blabel%3D%22gini%20%3D%200.147%5Cnsamples%20%3D%2025%5Cnvalue%20%3D%20%5B2%2C%2023%5D%22%5D%20%3B%0D%0A475%20-%3E%20476%20%3B%0D%0A477%20%5Blabel%3D%22gini%20%3D%200.455%5Cnsamples%20%3D%2020%5Cnvalue%20%3D%20%5B7%2C%2013%5D%22%5D%20%3B%0D%0A475%20-%3E%20477%20%3B%0D%0A478%20%5Blabel%3D%22day%20%3C%3D%2025.5%5Cngini%20%3D%200.243%5Cnsamples%20%3D%20120%5Cnvalue%20%3D%20%5B17%2C%20103%5D%22%5D%20%3B%0D%0A464%20-%3E%20478%20%3B%0D%0A479%20%5Blabel%3D%22marital_married%20%3C%3D%200.5%5Cngini%20%3D%200.131%5Cnsamples%20%3D%2057%5Cnvalue%20%3D%20%5B4%2C%2053%5D%22%5D%20%3B%0D%0A478%20-%3E%20479%20%3B%0D%0A480%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2029%5Cnvalue%20%3D%20%5B0%2C%2029%5D%22%5D%20%3B%0D%0A479%20-%3E%20480%20%3B%0D%0A481%20%5Blabel%3D%22gini%20%3D%200.245%5Cnsamples%20%3D%2028%5Cnvalue%20%3D%20%5B4%2C%2024%5D%22%5D%20%3B%0D%0A479%20-%3E%20481%20%3B%0D%0A482%20%5Blabel%3D%22duration%20%3C%3D%20691.5%5Cngini%20%3D%200.328%5Cnsamples%20%3D%2063%5Cnvalue%20%3D%20%5B13%2C%2050%5D%22%5D%20%3B%0D%0A478%20-%3E%20482%20%3B%0D%0A483%20%5Blabel%3D%22gini%20%3D%200.424%5Cnsamples%20%3D%2036%5Cnvalue%20%3D%20%5B11%2C%2025%5D%22%5D%20%3B%0D%0A482%20-%3E%20483%20%3B%0D%0A484%20%5Blabel%3D%22gini%20%3D%200.137%5Cnsamples%20%3D%2027%5Cnvalue%20%3D%20%5B2%2C%2025%5D%22%5D%20%3B%0D%0A482%20-%3E%20484%20%3B%0D%0A485%20%5Blabel%3D%22gini%20%3D%200.497%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B6%2C%207%5D%22%5D%20%3B%0D%0A449%20-%3E%20485%20%3B%0D%0A486%20%5Blabel%3D%22age%20%3C%3D%2033.5%5Cngini%20%3D%200.042%5Cnsamples%20%3D%2047%5Cnvalue%20%3D%20%5B1%2C%2046%5D%22%5D%20%3B%0D%0A448%20-%3E%20486%20%3B%0D%0A487%20%5Blabel%3D%22gini%20%3D%200.142%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B1%2C%2012%5D%22%5D%20%3B%0D%0A486%20-%3E%20487%20%3B%0D%0A488%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2034%5Cnvalue%20%3D%20%5B0%2C%2034%5D%22%5D%20%3B%0D%0A486%20-%3E%20488%20%3B%0D%0A489%20%5Blabel%3D%22age%20%3C%3D%2032.5%5Cngini%20%3D%200.432%5Cnsamples%20%3D%20111%5Cnvalue%20%3D%20%5B35%2C%2076%5D%22%5D%20%3B%0D%0A447%20-%3E%20489%20%3B%0D%0A490%20%5Blabel%3D%22gini%20%3D%200.18%5Cnsamples%20%3D%2030%5Cnvalue%20%3D%20%5B3%2C%2027%5D%22%5D%20%3B%0D%0A489%20-%3E%20490%20%3B%0D%0A491%20%5Blabel%3D%22marital_single%20%3C%3D%200.5%5Cngini%20%3D%200.478%5Cnsamples%20%3D%2081%5Cnvalue%20%3D%20%5B32%2C%2049%5D%22%5D%20%3B%0D%0A489%20-%3E%20491%20%3B%0D%0A492%20%5Blabel%3D%22duration%20%3C%3D%20716.5%5Cngini%20%3D%200.441%5Cnsamples%20%3D%2061%5Cnvalue%20%3D%20%5B20%2C%2041%5D%22%5D%20%3B%0D%0A491%20-%3E%20492%20%3B%0D%0A493%20%5Blabel%3D%22month%20%3C%3D%205.5%5Cngini%20%3D%200.369%5Cnsamples%20%3D%2041%5Cnvalue%20%3D%20%5B10%2C%2031%5D%22%5D%20%3B%0D%0A492%20-%3E%20493%20%3B%0D%0A494%20%5Blabel%3D%22gini%20%3D%200.454%5Cnsamples%20%3D%2023%5Cnvalue%20%3D%20%5B8%2C%2015%5D%22%5D%20%3B%0D%0A493%20-%3E%20494%20%3B%0D%0A495%20%5Blabel%3D%22gini%20%3D%200.198%5Cnsamples%20%3D%2018%5Cnvalue%20%3D%20%5B2%2C%2016%5D%22%5D%20%3B%0D%0A493%20-%3E%20495%20%3B%0D%0A496%20%5Blabel%3D%22gini%20%3D%200.5%5Cnsamples%20%3D%2020%5Cnvalue%20%3D%20%5B10%2C%2010%5D%22%5D%20%3B%0D%0A492%20-%3E%20496%20%3B%0D%0A497%20%5Blabel%3D%22gini%20%3D%200.48%5Cnsamples%20%3D%2020%5Cnvalue%20%3D%20%5B12%2C%208%5D%22%5D%20%3B%0D%0A491%20-%3E%20497%20%3B%0D%0A498%20%5Blabel%3D%22age%20%3C%3D%2064.0%5Cngini%20%3D%200.177%5Cnsamples%20%3D%20937%5Cnvalue%20%3D%20%5B92%2C%20845%5D%22%5D%20%3B%0D%0A446%20-%3E%20498%20%3B%0D%0A499%20%5Blabel%3D%22marital_married%20%3C%3D%200.5%5Cngini%20%3D%200.169%5Cnsamples%20%3D%20913%5Cnvalue%20%3D%20%5B85%2C%20828%5D%22%5D%20%3B%0D%0A498%20-%3E%20499%20%3B%0D%0A500%20%5Blabel%3D%22poutcome_other%20%3C%3D%200.5%5Cngini%20%3D%200.128%5Cnsamples%20%3D%20421%5Cnvalue%20%3D%20%5B29%2C%20392%5D%22%5D%20%3B%0D%0A499%20-%3E%20500%20%3B%0D%0A501%20%5Blabel%3D%22duration%20%3C%3D%201245.0%5Cngini%20%3D%200.116%5Cnsamples%20%3D%20403%5Cnvalue%20%3D%20%5B25%2C%20378%5D%22%5D%20%3B%0D%0A500%20-%3E%20501%20%3B%0D%0A502%20%5Blabel%3D%22contact_unknown%20%3C%3D%200.5%5Cngini%20%3D%200.14%5Cnsamples%20%3D%20304%5Cnvalue%20%3D%20%5B23%2C%20281%5D%22%5D%20%3B%0D%0A501%20-%3E%20502%20%3B%0D%0A503%20%5Blabel%3D%22balance%20%3C%3D%20402.5%5Cngini%20%3D%200.113%5Cnsamples%20%3D%20232%5Cnvalue%20%3D%20%5B14%2C%20218%5D%22%5D%20%3B%0D%0A502%20-%3E%20503%20%3B%0D%0A504%20%5Blabel%3D%22duration%20%3C%3D%20859.0%5Cngini%20%3D%200.038%5Cnsamples%20%3D%20103%5Cnvalue%20%3D%20%5B2%2C%20101%5D%22%5D%20%3B%0D%0A503%20-%3E%20504%20%3B%0D%0A505%20%5Blabel%3D%22gini%20%3D%200.172%5Cnsamples%20%3D%2021%5Cnvalue%20%3D%20%5B2%2C%2019%5D%22%5D%20%3B%0D%0A504%20-%3E%20505%20%3B%0D%0A506%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2082%5Cnvalue%20%3D%20%5B0%2C%2082%5D%22%5D%20%3B%0D%0A504%20-%3E%20506%20%3B%0D%0A507%20%5Blabel%3D%22balance%20%3C%3D%20724.0%5Cngini%20%3D%200.169%5Cnsamples%20%3D%20129%5Cnvalue%20%3D%20%5B12%2C%20117%5D%22%5D%20%3B%0D%0A503%20-%3E%20507%20%3B%0D%0A508%20%5Blabel%3D%22gini%20%3D%200.366%5Cnsamples%20%3D%2029%5Cnvalue%20%3D%20%5B7%2C%2022%5D%22%5D%20%3B%0D%0A507%20-%3E%20508%20%3B%0D%0A509%20%5Blabel%3D%22campaign%20%3C%3D%202.5%5Cngini%20%3D%200.095%5Cnsamples%20%3D%20100%5Cnvalue%20%3D%20%5B5%2C%2095%5D%22%5D%20%3B%0D%0A507%20-%3E%20509%20%3B%0D%0A510%20%5Blabel%3D%22pdays%20%3C%3D%20111.0%5Cngini%20%3D%200.031%5Cnsamples%20%3D%2064%5Cnvalue%20%3D%20%5B1%2C%2063%5D%22%5D%20%3B%0D%0A509%20-%3E%20510%20%3B%0D%0A511%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2051%5Cnvalue%20%3D%20%5B0%2C%2051%5D%22%5D%20%3B%0D%0A510%20-%3E%20511%20%3B%0D%0A512%20%5Blabel%3D%22gini%20%3D%200.142%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B1%2C%2012%5D%22%5D%20%3B%0D%0A510%20-%3E%20512%20%3B%0D%0A513%20%5Blabel%3D%22gini%20%3D%200.198%5Cnsamples%20%3D%2036%5Cnvalue%20%3D%20%5B4%2C%2032%5D%22%5D%20%3B%0D%0A509%20-%3E%20513%20%3B%0D%0A514%20%5Blabel%3D%22age%20%3C%3D%2033.5%5Cngini%20%3D%200.219%5Cnsamples%20%3D%2072%5Cnvalue%20%3D%20%5B9%2C%2063%5D%22%5D%20%3B%0D%0A502%20-%3E%20514%20%3B%0D%0A515%20%5Blabel%3D%22gini%20%3D%200.403%5Cnsamples%20%3D%2025%5Cnvalue%20%3D%20%5B7%2C%2018%5D%22%5D%20%3B%0D%0A514%20-%3E%20515%20%3B%0D%0A516%20%5Blabel%3D%22housing%20%3C%3D%200.5%5Cngini%20%3D%200.081%5Cnsamples%20%3D%2047%5Cnvalue%20%3D%20%5B2%2C%2045%5D%22%5D%20%3B%0D%0A514%20-%3E%20516%20%3B%0D%0A517%20%5Blabel%3D%22gini%20%3D%200.198%5Cnsamples%20%3D%2018%5Cnvalue%20%3D%20%5B2%2C%2016%5D%22%5D%20%3B%0D%0A516%20-%3E%20517%20%3B%0D%0A518%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2029%5Cnvalue%20%3D%20%5B0%2C%2029%5D%22%5D%20%3B%0D%0A516%20-%3E%20518%20%3B%0D%0A519%20%5Blabel%3D%22month%20%3C%3D%206.5%5Cngini%20%3D%200.04%5Cnsamples%20%3D%2099%5Cnvalue%20%3D%20%5B2%2C%2097%5D%22%5D%20%3B%0D%0A501%20-%3E%20519%20%3B%0D%0A520%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2056%5Cnvalue%20%3D%20%5B0%2C%2056%5D%22%5D%20%3B%0D%0A519%20-%3E%20520%20%3B%0D%0A521%20%5Blabel%3D%22balance%20%3C%3D%20414.0%5Cngini%20%3D%200.089%5Cnsamples%20%3D%2043%5Cnvalue%20%3D%20%5B2%2C%2041%5D%22%5D%20%3B%0D%0A519%20-%3E%20521%20%3B%0D%0A522%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2020%5Cnvalue%20%3D%20%5B0%2C%2020%5D%22%5D%20%3B%0D%0A521%20-%3E%20522%20%3B%0D%0A523%20%5Blabel%3D%22gini%20%3D%200.159%5Cnsamples%20%3D%2023%5Cnvalue%20%3D%20%5B2%2C%2021%5D%22%5D%20%3B%0D%0A521%20-%3E%20523%20%3B%0D%0A524%20%5Blabel%3D%22gini%20%3D%200.346%5Cnsamples%20%3D%2018%5Cnvalue%20%3D%20%5B4%2C%2014%5D%22%5D%20%3B%0D%0A500%20-%3E%20524%20%3B%0D%0A525%20%5Blabel%3D%22pdays%20%3C%3D%20345.5%5Cngini%20%3D%200.202%5Cnsamples%20%3D%20492%5Cnvalue%20%3D%20%5B56%2C%20436%5D%22%5D%20%3B%0D%0A499%20-%3E%20525%20%3B%0D%0A526%20%5Blabel%3D%22pdays%20%3C%3D%207.5%5Cngini%20%3D%200.194%5Cnsamples%20%3D%20479%5Cnvalue%20%3D%20%5B52%2C%20427%5D%22%5D%20%3B%0D%0A525%20-%3E%20526%20%3B%0D%0A527%20%5Blabel%3D%22duration%20%3C%3D%20890.0%5Cngini%20%3D%200.216%5Cnsamples%20%3D%20413%5Cnvalue%20%3D%20%5B51%2C%20362%5D%22%5D%20%3B%0D%0A526%20-%3E%20527%20%3B%0D%0A528%20%5Blabel%3D%22balance%20%3C%3D%20968.0%5Cngini%20%3D%200.32%5Cnsamples%20%3D%2085%5Cnvalue%20%3D%20%5B17%2C%2068%5D%22%5D%20%3B%0D%0A527%20-%3E%20528%20%3B%0D%0A529%20%5Blabel%3D%22job_blue-collar%20%3C%3D%200.5%5Cngini%20%3D%200.417%5Cnsamples%20%3D%2054%5Cnvalue%20%3D%20%5B16%2C%2038%5D%22%5D%20%3B%0D%0A528%20-%3E%20529%20%3B%0D%0A530%20%5Blabel%3D%22month%20%3C%3D%207.5%5Cngini%20%3D%200.326%5Cnsamples%20%3D%2039%5Cnvalue%20%3D%20%5B8%2C%2031%5D%22%5D%20%3B%0D%0A529%20-%3E%20530%20%3B%0D%0A531%20%5Blabel%3D%22gini%20%3D%200.204%5Cnsamples%20%3D%2026%5Cnvalue%20%3D%20%5B3%2C%2023%5D%22%5D%20%3B%0D%0A530%20-%3E%20531%20%3B%0D%0A532%20%5Blabel%3D%22gini%20%3D%200.473%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B5%2C%208%5D%22%5D%20%3B%0D%0A530%20-%3E%20532%20%3B%0D%0A533%20%5Blabel%3D%22gini%20%3D%200.498%5Cnsamples%20%3D%2015%5Cnvalue%20%3D%20%5B8%2C%207%5D%22%5D%20%3B%0D%0A529%20-%3E%20533%20%3B%0D%0A534%20%5Blabel%3D%22gini%20%3D%200.062%5Cnsamples%20%3D%2031%5Cnvalue%20%3D%20%5B1%2C%2030%5D%22%5D%20%3B%0D%0A528%20-%3E%20534%20%3B%0D%0A535%20%5Blabel%3D%22balance%20%3C%3D%202120.5%5Cngini%20%3D%200.186%5Cnsamples%20%3D%20328%5Cnvalue%20%3D%20%5B34%2C%20294%5D%22%5D%20%3B%0D%0A527%20-%3E%20535%20%3B%0D%0A536%20%5Blabel%3D%22job_technician%20%3C%3D%200.5%5Cngini%20%3D%200.146%5Cnsamples%20%3D%20253%5Cnvalue%20%3D%20%5B20%2C%20233%5D%22%5D%20%3B%0D%0A535%20-%3E%20536%20%3B%0D%0A537%20%5Blabel%3D%22balance%20%3C%3D%201464.5%5Cngini%20%3D%200.162%5Cnsamples%20%3D%20225%5Cnvalue%20%3D%20%5B20%2C%20205%5D%22%5D%20%3B%0D%0A536%20-%3E%20537%20%3B%0D%0A538%20%5Blabel%3D%22balance%20%3C%3D%2045.5%5Cngini%20%3D%200.18%5Cnsamples%20%3D%20200%5Cnvalue%20%3D%20%5B20%2C%20180%5D%22%5D%20%3B%0D%0A537%20-%3E%20538%20%3B%0D%0A539%20%5Blabel%3D%22duration%20%3C%3D%201345.0%5Cngini%20%3D%200.074%5Cnsamples%20%3D%2052%5Cnvalue%20%3D%20%5B2%2C%2050%5D%22%5D%20%3B%0D%0A538%20-%3E%20539%20%3B%0D%0A540%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2039%5Cnvalue%20%3D%20%5B0%2C%2039%5D%22%5D%20%3B%0D%0A539%20-%3E%20540%20%3B%0D%0A541%20%5Blabel%3D%22gini%20%3D%200.26%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B2%2C%2011%5D%22%5D%20%3B%0D%0A539%20-%3E%20541%20%3B%0D%0A542%20%5Blabel%3D%22balance%20%3C%3D%20401.0%5Cngini%20%3D%200.214%5Cnsamples%20%3D%20148%5Cnvalue%20%3D%20%5B18%2C%20130%5D%22%5D%20%3B%0D%0A538%20-%3E%20542%20%3B%0D%0A543%20%5Blabel%3D%22age%20%3C%3D%2037.0%5Cngini%20%3D%200.303%5Cnsamples%20%3D%2059%5Cnvalue%20%3D%20%5B11%2C%2048%5D%22%5D%20%3B%0D%0A542%20-%3E%20543%20%3B%0D%0A544%20%5Blabel%3D%22gini%20%3D%200.083%5Cnsamples%20%3D%2023%5Cnvalue%20%3D%20%5B1%2C%2022%5D%22%5D%20%3B%0D%0A543%20-%3E%20544%20%3B%0D%0A545%20%5Blabel%3D%22gini%20%3D%200.401%5Cnsamples%20%3D%2036%5Cnvalue%20%3D%20%5B10%2C%2026%5D%22%5D%20%3B%0D%0A543%20-%3E%20545%20%3B%0D%0A546%20%5Blabel%3D%22age%20%3C%3D%2054.5%5Cngini%20%3D%200.145%5Cnsamples%20%3D%2089%5Cnvalue%20%3D%20%5B7%2C%2082%5D%22%5D%20%3B%0D%0A542%20-%3E%20546%20%3B%0D%0A547%20%5Blabel%3D%22month%20%3C%3D%206.5%5Cngini%20%3D%200.083%5Cnsamples%20%3D%2069%5Cnvalue%20%3D%20%5B3%2C%2066%5D%22%5D%20%3B%0D%0A546%20-%3E%20547%20%3B%0D%0A548%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2040%5Cnvalue%20%3D%20%5B0%2C%2040%5D%22%5D%20%3B%0D%0A547%20-%3E%20548%20%3B%0D%0A549%20%5Blabel%3D%22gini%20%3D%200.185%5Cnsamples%20%3D%2029%5Cnvalue%20%3D%20%5B3%2C%2026%5D%22%5D%20%3B%0D%0A547%20-%3E%20549%20%3B%0D%0A550%20%5Blabel%3D%22gini%20%3D%200.32%5Cnsamples%20%3D%2020%5Cnvalue%20%3D%20%5B4%2C%2016%5D%22%5D%20%3B%0D%0A546%20-%3E%20550%20%3B%0D%0A551%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2025%5Cnvalue%20%3D%20%5B0%2C%2025%5D%22%5D%20%3B%0D%0A537%20-%3E%20551%20%3B%0D%0A552%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2028%5Cnvalue%20%3D%20%5B0%2C%2028%5D%22%5D%20%3B%0D%0A536%20-%3E%20552%20%3B%0D%0A553%20%5Blabel%3D%22balance%20%3C%3D%202599.5%5Cngini%20%3D%200.304%5Cnsamples%20%3D%2075%5Cnvalue%20%3D%20%5B14%2C%2061%5D%22%5D%20%3B%0D%0A535%20-%3E%20553%20%3B%0D%0A554%20%5Blabel%3D%22gini%20%3D%200.48%5Cnsamples%20%3D%2015%5Cnvalue%20%3D%20%5B6%2C%209%5D%22%5D%20%3B%0D%0A553%20-%3E%20554%20%3B%0D%0A555%20%5Blabel%3D%22age%20%3C%3D%2036.5%5Cngini%20%3D%200.231%5Cnsamples%20%3D%2060%5Cnvalue%20%3D%20%5B8%2C%2052%5D%22%5D%20%3B%0D%0A553%20-%3E%20555%20%3B%0D%0A556%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2017%5Cnvalue%20%3D%20%5B0%2C%2017%5D%22%5D%20%3B%0D%0A555%20-%3E%20556%20%3B%0D%0A557%20%5Blabel%3D%22education_secondary%20%3C%3D%200.5%5Cngini%20%3D%200.303%5Cnsamples%20%3D%2043%5Cnvalue%20%3D%20%5B8%2C%2035%5D%22%5D%20%3B%0D%0A555%20-%3E%20557%20%3B%0D%0A558%20%5Blabel%3D%22gini%20%3D%200.165%5Cnsamples%20%3D%2022%5Cnvalue%20%3D%20%5B2%2C%2020%5D%22%5D%20%3B%0D%0A557%20-%3E%20558%20%3B%0D%0A559%20%5Blabel%3D%22gini%20%3D%200.408%5Cnsamples%20%3D%2021%5Cnvalue%20%3D%20%5B6%2C%2015%5D%22%5D%20%3B%0D%0A557%20-%3E%20559%20%3B%0D%0A560%20%5Blabel%3D%22age%20%3C%3D%2052.5%5Cngini%20%3D%200.03%5Cnsamples%20%3D%2066%5Cnvalue%20%3D%20%5B1%2C%2065%5D%22%5D%20%3B%0D%0A526%20-%3E%20560%20%3B%0D%0A561%20%5Blabel%3D%22gini%20%3D%200.0%5Cnsamples%20%3D%2053%5Cnvalue%20%3D%20%5B0%2C%2053%5D%22%5D%20%3B%0D%0A560%20-%3E%20561%20%3B%0D%0A562%20%5Blabel%3D%22gini%20%3D%200.142%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B1%2C%2012%5D%22%5D%20%3B%0D%0A560%20-%3E%20562%20%3B%0D%0A563%20%5Blabel%3D%22gini%20%3D%200.426%5Cnsamples%20%3D%2013%5Cnvalue%20%3D%20%5B4%2C%209%5D%22%5D%20%3B%0D%0A525%20-%3E%20563%20%3B%0D%0A564%20%5Blabel%3D%22gini%20%3D%200.413%5Cnsamples%20%3D%2024%5Cnvalue%20%3D%20%5B7%2C%2017%5D%22%5D%20%3B%0D%0A498%20-%3E%20564%20%3B%0D%0A%7D)

*4. Оцените важность признаков с помощью атрибута feature_importances_*

In [ ]:
features = dict(zip(range(len(X.columns)), X.columns))

# Важность признаков
importances = best_tree.feature_importances_

indices = np.argsort(importances)[::-1]
# Plot the feature importancies of the forest
num_to_plot = max(10, len(X.columns))
feature_indices = [ind for ind in indices[:num_to_plot]]

# Print the feature ranking
print("Feature ranking:")

for f in range(num_to_plot):
    print(f+1, features[feature_indices[f]], importances[indices[f]])

In [ ]:
# Plot the impurity-based feature importances of the forest
plt.figure(figsize=(15,15))
plt.title("Feature importances")
bars = plt.bar(range(num_to_plot), 
               importances[indices[:num_to_plot]],
               color=([str(i/float(num_to_plot+1)) for i in range(num_to_plot)]),
               align="center")

ticks = plt.xticks(range(num_to_plot), 
                   feature_indices)

plt.xlim([-1, num_to_plot])
plt.legend(bars, [u''.join(features[i]) for i in feature_indices]);

*5. Сформулируйте выводы.*

Самым влиятельным из гиперпараметров оказался max_depth.

Самым влиятельным признаком является "duration" (last contact duration, in seconds (numeric)).
Также в топ 3 влиятельных признаков входит: "poutcome_success", "month".
Признаки "job", "education", "martial" менее всего влиятельны.

# Случайный лес 

*1. Постройте модель случайного леса (Random Forest) для вашей задачи с гиперпараметрами по умолчанию и оцените качество модели на отложенной выборке.*

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#rf = RandomForestClassifier(n_estimators=100, random_state=2019, max_depth=6)

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_valid)

accuracy_score(y_valid, y_pred)

In [ ]:
rf_param = {'n_estimators': [50, 100, 200, 300]
            #, 'max_depth':[5, 6, 7, 8, 9, 10]
           }
new_rf = GridSearchCV(rf, rf_param, cv=5, scoring='accuracy')
new_rf.fit(X_train, y_train)

In [ ]:
new_rf.best_params_

In [ ]:
new_rf.best_estimator_

In [ ]:
new_rf.best_score_

*2. Осуществите подбор гиперпараметров модели:*
- *количество деревьев n_estimators;*
- *максимальная глубина дерева max_depth;*
- *минимальное число объектов для разбиения во внутренней вершине min_samples_split;*
- *минимальное число объектов в листе min_samples_leaf*
- *максимальное количество признаков, рассматриваемых при поиске лучшего разбиения max_features.*


In [ ]:
rf_params_n_estimators = {'n_estimators': [50, 100, 200, 300]}
rf_grid_n_estimators = GridSearchCV(rf, rf_params_n_estimators, cv=kf, scoring='accuracy', n_jobs = -1)
rf_grid_n_estimators.fit(X_train, y_train)

In [ ]:
print(rf_grid_n_estimators.best_score_, "\n", rf_grid_n_estimators.best_params_)

In [ ]:
rf = RandomForestClassifier(n_estimators = 300)
rf_params_max_depth = {'max_depth': np.arange(2, 38)}
rf_grid_max_depth = GridSearchCV(rf, rf_params_max_depth, cv=kf, scoring='accuracy', n_jobs = -1)
rf_grid_max_depth.fit(X_train, y_train)

In [ ]:
print(rf_grid_max_depth.best_estimator_,"\n", rf_grid_max_depth.best_score_, "\n", rf_grid_max_depth.best_params_)

In [ ]:
rf = RandomForestClassifier(n_estimators = 300, max_depth=18)
rf_params_min_samples_split = {'min_samples_split': np.arange(2, 38)}
rf_grid_min_samples_split = GridSearchCV(rf, rf_params_min_samples_split, cv=kf, scoring='accuracy', n_jobs = -1)
rf_grid_min_samples_split.fit(X_train, y_train)

In [ ]:
print(rf_grid_min_samples_split.best_estimator_,"\n", rf_grid_min_samples_split.best_score_, "\n", rf_grid_min_samples_split.best_params_)

In [ ]:
rf = RandomForestClassifier(n_estimators = 300, max_depth = 18, min_samples_split = 4)
rf_params_min_samples_leaf = {'min_samples_leaf': np.arange(2, 38)}
rf_grid_min_samples_leaf = GridSearchCV(rf, rf_params_min_samples_leaf, cv=kf, scoring='accuracy', n_jobs = -1)
rf_grid_min_samples_leaf.fit(X_train, y_train)

In [ ]:
print(rf_grid_min_samples_leaf.best_estimator_,"\n", rf_grid_min_samples_leaf.best_score_, "\n", rf_grid_min_samples_leaf.best_params_)

*3. Постройте валидационные кривые для каждого из пиперпараметров.*

In [ ]:
# Построим графики зависимости средней доли правильных ответов 
# (mean_test_score) от значений каждого из гиперпараметров.
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=2, sharey=True, figsize=(8,13))

ax[0, 0].plot(rf_params_n_estimators['n_estimators'], rf_grid_n_estimators.cv_results_['mean_test_score'])
ax[0, 0].set_xlabel('n_estimators')
ax[0, 0].set_ylabel('Mean accuracy on test set')

ax[0, 1].plot(rf_params_max_depth['max_depth'], rf_grid_max_depth.cv_results_['mean_test_score'])
ax[0, 1].set_xlabel('max_depth')
ax[0, 1].set_ylabel('Mean accuracy on test set')

ax[1, 0].plot(rf_params_min_samples_split['min_samples_split'], rf_grid_min_samples_split.cv_results_['mean_test_score'])
ax[1, 0].set_xlabel('min_samples_split')
ax[1, 0].set_ylabel('Mean accuracy on test set')

ax[1, 1].plot(rf_params_min_samples_leaf['min_samples_leaf'], rf_grid_min_samples_leaf.cv_results_['mean_test_score'])
ax[1, 1].set_xlabel('min_samples_leaf')
ax[1, 1].set_ylabel('Mean accuracy on test set')

Результативный набор гиперпараметров: max_depth=18, min_samples_leaf=2, min_samples_split=4, n_estimators=300

*4. Оцените важность признаков в данной модели. Визуализируйте топ-10 самых полезных признаков с помощью столбчаатой диаграммы.*

In [ ]:
best_forest = RandomForestClassifier(max_depth = 18, 
                                     min_samples_leaf = 2, 
                                     min_samples_split = 4,
                                     n_estimators = 300)
best_forest.fit(X_train, y_train)

In [ ]:
importances = best_forest.feature_importances_

indices = np.argsort(importances)[::-1]
# Plot the feature importancies of the forest
num_to_plot = max(10, len(X.columns))
feature_indices = [ind for ind in indices[:num_to_plot]]

# Print the feature ranking
print("Feature ranking:")

for f in range(num_to_plot):
    print(f+1, features[feature_indices[f]], importances[indices[f]])

In [ ]:
# Plot the impurity-based feature importances of the forest
plt.figure(figsize=(15,15))
plt.title("Feature importances")
bars = plt.bar(range(num_to_plot), 
               importances[indices[:num_to_plot]],
               color=([str(i/float(num_to_plot+1)) for i in range(num_to_plot)]),
               align="center")

ticks = plt.xticks(range(num_to_plot), 
                   feature_indices)

plt.xlim([-1, num_to_plot])
plt.legend(bars, [u''.join(features[i]) for i in feature_indices]);

Как можно заметить самым влиятельным признаком остается "duration" (при этом в сравнении с предыдущим графиком значение (0.37) ниже). 

В топ 10 влиятельных признаков входят:
1. duration          0.3789128070804476
2. month             0.07598477982687375
3. age               0.07148947525673802
4. balance           0.06986031985587046
5. day               0.061987089153512316
6. poutcome_success  0.04304796307943662
7. pdays             0.03738148170779854
8. housing           0.03188747942140964
9. campaign          0.028810437904120415
10. contact_unknown  0.028508804186221825

In [ ]:
# Plot the impurity-based feature importances of the forest
plt.figure(figsize=(15,15))
plt.title("Feature importances")
bars = plt.bar(range(10), 
               importances[indices[:10]],
               color=([str(i/float(10+1)) for i in range(10)]),
               align="center")

ticks = plt.xticks(range(10), 
                   feature_indices)

plt.xlim([-1, num_to_plot])
plt.legend(bars, [u''.join(features[i]) for i in feature_indices]);

*5. Сравните результаты трёх моделей: метода ближайших соседей, дерева решений и случайного леса. Сформулируйте выводы.*

Подсумирую свои впечатления:
- Качество модели метода ближайших соседей оказался самым худшим и вычислительно сложным.
- Дерево решений было досаточно быстрым, модель оказалась лучше чем в случае kNN. Плюсом является простая в интерпретации визуализация модели.
- Случайный лес показал результат лучше чем kNN и относительно схожий результат с деревом решений. Минусом является длительность выполнения работы при большом количестве деревьев.